# CheckList

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from nltk.corpus import wordnet as wn

import checklist
import spacy
import itertools
import pandas as pd

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

## Setting CheckList utilities

" spaCy is an open-source software library for advanced natural language processing: https://github.com/explosion/spaCy. spaCy comes with pretrained statistical models and word vectors, and currently supports tokenization for 60+ languages. It features state-of-the-art speed, convolutional neural network models for tagging, parsing and named entity recognition and easy deep learning integration. "

In [2]:
nlp = spacy.load('en_core_web_sm')

/Users/Marta/opt/anaconda3/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


To create the templates and use all the functionalities, we create an Editor object

In [3]:
editor = checklist.editor.Editor()
editor.tg

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating a new object, TestSuite, where we'll add all the custom tests, inspired by the original notebooks plus new ones

In [4]:
suite = TestSuite()

## Exploring the linguistic data and tools available to build/expand lexicons and tests

Important words or word types for the task

### HurtLex

**HurtLex**

From the paper, section 4.2 Misogyny Identification on Social Media:
_They identified the Prostitution, Female and Male Sexual Apparatus and
Physical and Mental Diversity and Disability categories as the most informative for this task._ The task they are talking about is precisely AMI

In [5]:
hurtlex=pd.read_csv('/Users/Marta/CheckList - FBK/hurtlex/lexica/EN/1.2/hurtlex_EN.tsv', sep='\t', index_col=None, header=0)

In building the lexica, we filter out per POS = n, a, v, av 

Selecting the relevant categories: PR (words related to prostitution), ASM (male genitalia), ASF (female genitalia), DDF (physical disabilities and diversity), DDP (cognitive disabilities and diversity), OM (Homosexuality), QAS	(with potential negative connotations), CDS	(derogatory words)

In [6]:
hurtlex_AMI_pr=hurtlex[(hurtlex.category=='pr')]
hurtlex_AMI_asm=hurtlex[(hurtlex.category=='asm')]
hurtlex_AMI_asf=hurtlex[(hurtlex.category=='asf')]
hurtlex_AMI_dis=hurtlex[(hurtlex.category=='ddf')]
hurtlex_AMI_dis=hurtlex_AMI_dis.append(hurtlex[(hurtlex.category=='ddp')])
hurtlex_AMI_om=hurtlex[(hurtlex.category=='om')]
hurtlex_AMI_off=hurtlex[(hurtlex.category=='qas')]
hurtlex_AMI_off=hurtlex_AMI_off.append([(hurtlex.category=='cds')])

### Expanding the lexicons

Exploring the lexicons.keys

In [7]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from']


_Names_

Some of the offensive terms taken from: 
* https://www.macmillandictionary.com/thesaurus-category/british/insulting-words-for-a-woman
* http://www.slate.com/blogs/lexicon_valley/2013/09/11/top_swear_words_most_popular_curse_words_on_facebook.html
* https://github.com/minerva-ml/open-solution-toxic-comments/blob/master/external_data/compiled_bad_words.txt
* https://github.com/RobertJGabriel/Google-profanity-words/blob/master/list.txt

In [8]:
# kind of neutral, taken by identity terms' file within AMI2020's data
women_noun = ['woman', 'sister', 'daughter', 'girl', 'wife', 'young lady', 'lady',
              'grandmother', 'mum', 'girlfriend', 'partner']#, 'bride', 'housewife', Mrs', 'Miss']

# We don't need plural in templates for now
#women_noun_plural = ['women', 'sisters', 'aunts', 'daughters', 'girls', 'mothers', 'wives', 'young ladies', 'ladies', 'brides', 
#              'grandmothers', 'mums', 'housewives', 'females', 'girlfriends', 'they']

# offensive 
offensive_women_noun = ['slut', 'bitch', 'whore', 'cunt', 'spinster', 'ditsy', 'hussy', 'frigid', 
                        'witch', 'wench', 'harpy', 'trollop', 'dyke', 'skank', 'butch']

# We don't need plural in templates for now
#offensive_women_noun_plural = ['sluts', 'bitches', 'whores', 'cunts']

offensive_homosexuals = []
for term in hurtlex_AMI_om[(hurtlex_AMI_om.pos=='n')&(hurtlex_AMI_om.level=='conservative')].lemma:
    offensive_homosexuals.append(term)

offensive_generic = ['twat', 'prick', 'motherfucker', 'shit', 'shithead', 'dick', 'dickhead', 
                     'cock', 'cockface', 'cockhead', 'cocksucker', 'pussy', 'asshole', 'arsehole', 'assfucker', 
                     'fag', 'faggot', 'bastard', 'douche', 'bugger']

In [9]:
dis = ['weirdo', 'lame', 'cripple', 'invalid', 'inferm'] #'disabled', 'handicapped'
homeless = ['tramp', 'hobo', 'drifter', 'beggar', 'vagrant']#'roofless', 'homeless', 'houseless'
old = ['codger', 'grandpa', 'oldie', 'geezer', 'elder']

In [10]:
work_role = ['astronaut', 'academic', 'accountant', 'activist', 'administrator', 'advisor', 'agent', 'analyst','architect', 
             'assistant', 'attorney', 'artist' 'boxer', 'builder', 'businessman', 'cameraman', 'carpenter', 'clown', 
             'consultant', 'dentist', 'detective', 'developer', 'doctor', 'doorman', 'driver', 'electrician', 
             'economist', 'editor', 'educator', 'entrepreneur', 'executive',
             'engineer', 'farmer',  'fighter', 'fireman', 'footballer', 'foreman', 
             'historian', 'hitter', 'intern', 'interpreter', 'investigator', 'investor', 'gardener', 'housekeeper', 
             'journalist', 'librarian', 'lifeguard', 'magician', 'mechanic', 'nun', 'nurse', 'painter', 'paramedic', 
             'photographer', 'pilot', 'police', 'poet', 'postman', 'priest', 'professor', 'ranger', 'repairman', 'reporter', 
             'salesman', 'scientist', 'secretary', 'singer', 'soldier', 'student', 'surgeon', 'teacher', 'waiter', 'writer', 
             'attendant', 'officer', 'player', 'organizer', 'quarterback', 'shooter']

# These sterotyped lists are taken from the file male_occupations.txt and female_occupations.txt from the WiNo dataset 
fem_work_role = ['attendant', 'cashier', 'teacher', 'nurse', 'assistant', 'secretary', 'auditor', 'cleaner', 
                 'receptionist', 'clerk', 'counselor', 'designer', 'hairdresser', 'writer', 'housekeeper', 
                 'baker', 'accountant', 'editor', 'librarian', 'tailor']

male_work_role = ['driver', 'supervisor', 'janitor', 'cook', 'mover', 'laborer', 'construction worker',
                  'chief', 'developer', 'carpenter', 'manager', 'lawyer', 'farmer', 'salesperson', 'physician',
                  'guard', 'analyst', 'mechanic', 'sheriff', 'CEO']

In [11]:
editor.add_lexicon('women_noun', women_noun, overwrite=True)
editor.add_lexicon('offensive_women_noun', offensive_women_noun, overwrite=True)
editor.add_lexicon('offensive_homosexuals', offensive_homosexuals, overwrite=True)
editor.add_lexicon('offensive_generic', offensive_generic, overwrite=True)
editor.add_lexicon('dis', dis, overwrite=True)
editor.add_lexicon('homeless', homeless, overwrite=True)
editor.add_lexicon('old', old, overwrite=True)
editor.add_lexicon('work_role', work_role, overwrite=True)
editor.add_lexicon('fem_work_role', fem_work_role, overwrite=True)
editor.add_lexicon('male_work_role', male_work_role, overwrite=True)

_Adjectives_

In [12]:
print(', '.join(editor.suggest('She is {a:mask} {women_noun}.')[:100]))

amazing, extraordinary, old, incredible, older, excellent, exceptional, important, exemplary, American, outstanding, interesting, awesome, ordinary, unusual, ideal, great, honest, awful, beautiful, only, evil, lovely, emotional, odd, average, unbelievable, angry, Italian, unmarried, outspoken, elderly, elegant, admirable, good, inspiring, influential, honorable, elder, impressive, ambitious, wonderful, independent, active, enormous, eccentric, English, Indian, innocent, aging, Irish, unhappy, experienced, imposing, ancient, ugly, nice, understanding, African, artistic, unconventional, adventurous, young, strong, fantastic, happy, real, intelligent, terrific, accomplished, honorary, attractive, sweet, successful, educated, tough, actual, astonishing, absolute, uncommon, unexpected, inspirational, exquisite, remarkable, eminent, open, unfortunate, aggressive, alcoholic, imperfect, illustrious, absent, unforgettable, brilliant, unlikely, easy, astounding, invisible, iconic, single


In [13]:
print(', '.join(editor.suggest('She is {a:mask} {offensive_women_noun}.')[:100]))

fucking, evil, absolute, little, true, old, beautiful, ugly, actual, amazing, awful, excellent, incredible, arrogant, angry, enormous, utter, awesome, common, emotional, real, incompetent, abusive, insane, ignorant, American, adorable, obvious, idiot, wicked, ancient, good, great, total, attractive, pretty, adult, honest, complete, elderly, anonymous, bad, eternal, terrible, black, white, powerful, horrible, big, dead, nasty, dirty, perfect, young, damn, proud, annoying, strange, asshole, insecure, alcoholic, ultimate, extreme, immature, aggressive, animal, average, appalling, educated, older, easy, infamous, accomplished, adolescent, exquisite, odd, expert, Italian, Irish, innocent, intellectual, dangerous, active, vicious, aging, arch, anarchist, effective, female, master, original, mad, former, walking, lovely, dark, murderous, bloody, sex, blue


Some of the offensive adj are taken from:
* theodysseyonline.com/16-derogatory-words-used-to-describe-women

In [14]:
pos_adj = [                                                                             
    'adorable', 'amazing', 'astonishing', 'attractive', 'awesome', 'beautiful',
    'brilliant', 'caring', 'committed', 'cool', 'enthusiastic', 'excellent', 'exceptional',
    'extraordinary', 'fantastic', 'fun', 'good', 'great', 'happy', 'honest', 'impressive', 'incredible',
    'inspiring', 'intelligent', 'lovely', 'nice', 
    'successful', 'sweet', 'trustworthy', 'wise', 'wonderful']

neg_adj = [
    'absolute', 'aggressive', 'angry', 'annoying', 'arrogant', 
    'awful', 'bad', 'bitchy', 'bloody', 'boring', 'bossy', 'creepy', 'cruel', 
    'damn', 'dead', 'depressing', 'difficult', 'dirty', 'disappointing', 'disgusting', 
    'dreadful', 'emotional', 'evil', 'fake', 'fat', 'feisty', 'frustrating', 'fucking', 
    'giant', 'hormonal', 'horrible', 'hysterical', 'illogical', 'incompetent', 
    'irrational', 'irritating', 'lame', 'lousy', 'lying', 'nasty', 'poor', 
    'rough', 'sad', 'sassy', 'shameless', 'shitty', 'stupid', 'terrible', 'terrific', 
    'twatty', 'ugly', 'unhappy', 'wanky', 'weak', 'weird', 'whining', 'wicked', 'worst']

neutral_adj = [                                           
    'american', 'armenian','atheist', 'australian', 'black', 'british', 'christian', 'commercial', 
    'english', 'gay', 'immigrant', 'independent', 'independent', 'indian', 'indian', 'intellectual','busy', 
    'international', 'israeli', 'italian', 'jewish', 'married', 'muslim', 'old', 'poor',
    'private', 'rich', 'russian', 'single', 'tall', 'unmarried', 'white', 'working', 'young']

editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)

_Verbs_

In [15]:
print(', '.join(editor.suggest('I really {mask} {women_noun}.')[:100]))

like, miss, liked, love, mean, missed, hate, do, loved, am, appreciate, admire, respect, enjoy, did, enjoyed, dislike, want, wanted, feel, need, dig, was, care, believe, prefer, appreciated, think, have, felt, meant, know, hated, thought, see, hurt, admired, adore, needed, understand, value, got, disliked, trust, despise, thank, wish, hope, tried, fancy, the, heart, cherish, fell, married, lost, my, underestimated, blame, remember, envy, valued, resent, consider, fucking, get, a, sorry, helped, help, forgot, that, respected, cherished, say, call, had, support, called, understood, met, believed, preferred, are, likes, is, regret, trusted, wanna, missing, should, suck, worry, will, said, considered, try, liking, must, adopted


In [16]:
print(', '.join(editor.suggest('I really {mask} {offensive_women_noun}.')[:100]))

like, hate, liked, do, mean, dislike, love, am, did, miss, hated, enjoy, feel, want, was, despise, dig, prefer, enjoyed, fucking, wanted, appreciate, got, felt, disliked, think, need, wanna, get, a, meant, believe, loved, the, is, rather, missed, say, can, respect, resent, admire, understand, really, just, thought, fancy, suck, needed, know, preferred, have, hope, tried, blame, likes, go, LIKE, should, could, will, no, would, must, are, play, gotta, appreciated, adore, quite, bad, played, heart, said, don, lost, cannot, see, my, that, dont, read, trust, try, use, hurt, became, hit, dug, LOVE, fear, care, considered, smell, value, wish, fuck, regret, Like, never


In [17]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love', 'admire',
                   'respect', 'adore', 'support', 'care for', 'fancy', 'treasure', 'trust']

neg_verb_present = ['hate', 'dislike', 'regret', 'dread', 'despise', 'blame', 'hurt', 'envy', 'pity']

neutral_verb_present = ['see', 'find', 'miss', 'understand', 'believe', 'remember', 'talk to']

pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 
                 'respected', 'adored', 'supported', 'cared for', 'treasured', 'trusted']

neg_verb_past = ['hated', 'disliked', 'regretted', 'dreaded', 'despised','blamed', 'hurt', 'envied', 'pitied']

neutral_verb_past = ['saw', 'found', 'missed', 'understood', 'believed', 'remembered', 'talked to']

editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

_Intensifiers and reducers_

In [18]:
print(' , '.join(editor.suggest('{it} {be} {a:mask} {pos_adj} {women_noun}.', it=['She'], be=['is', 'was'])[:50]))

very , really , absolutely , extremely , incredibly , truly , pretty , genuinely , totally , exceptionally , wonderfully , quite , amazingly , just , remarkably , extraordinarily , utterly , unbelievably , especially , completely , exceedingly , insanely , super , enormously , unusually , equally , seriously , obviously , immensely , amazing , most , overall , an , otherwise , incredible , undeniably , awfully , awesome , entirely , extraordinary , real , almost , damn , actually , rather , highly , unexpectedly , altogether , actual , apparently


In [19]:
print(' , '.join(editor.suggest('{it} {be} {a:mask} {neg_adj} {offensive_women_noun}.', it=['She'], be=['is', 'was'])[:50]))

really , fucking , incredibly , extremely , very , absolutely , absolute , utterly , especially , equally , exceptionally , exceedingly , unbelievably , extraordinarily , truly , awfully , awful , bloody , amazingly , immensely , entirely , obviously , enormously , pretty , ugly , increasingly , insanely , actual , total , utter , obvious , outright , emotionally , almost , undeniably , big , intensely , even , real , endlessly , enormous , rather , excellent , infinitely , admittedly , complete , totally , apparently , completely , goddamn


In [20]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'especially',
              'exceptionally', 'utterly', 'rather', 'totally', 'particularly',
              'remarkably', 'pretty', 'wonderfully', 'completely',
              'entirely', 'undeniably', 'highly']

editor.add_lexicon('intens_adj', intens_adj, overwrite=True)

In [21]:
print(', '.join(editor.suggest('{i} {mask} {pos_verb} {the} {women_noun}.', i=['I', 'We', 'You'], the=['this', 'that', 'the'])[:100]))

really, just, always, also, all, certainly, both, never, truly, definitely, actually, still, so, quite, absolutely, obviously, totally, genuinely, rather, greatly, clearly, thoroughly, probably, very, sure, simply, deeply, especially, already, even, seriously, particularly, completely, too, have, only, honestly, had, finally, kinda, desperately, much, almost, did, highly, were, mostly, generally, personally, strongly, immediately, do, guys, most, sincerely, fucking, instantly, REALLY, often, would, fully, dearly, surely, basically, hardly, are, secretly, ..., each, usually, ever, felt, must, should, immensely, apparently, literally, tremendously, first, sorely, pretty, we, somehow, badly, somewhat, more, might, two, feel, could, will, can, profoundly, now, ALL, initially, ultimately, naturally, better, barely


In [22]:
print(', '.join(editor.suggest('{i} {mask} {neg_verb} {the} {offensive_women_noun}.', i=['I', 'We', 'You'], the=['this', 'that', 'the'])[:100]))

always, all, really, just, also, still, absolutely, so, actually, simply, never, only, even, truly, already, totally, certainly, both, immediately, definitely, rather, too, completely, fucking, now, seriously, honestly, obviously, clearly, almost, had, deeply, have, probably, especially, secretly, generally, particularly, genuinely, mostly, personally, quite, often, then, usually, finally, dearly, thoroughly, instantly, once, greatly, sure, sincerely, strongly, ever, forever, basically, literally, collectively, therefore, bitterly, kinda, desperately, much, very, utterly, openly, highly, constantly, ultimately, silently, do, long, fully, must, first, most, ALL, each, merely, can, mainly, practically, two, people, men, equally, suddenly, should, ourselves, guys, sorely, later, did, sometimes, will, again, soon, would, specifically


In [23]:
intens_verb = ['really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 
               'highly', 'definitely', 'certainly', 'honestly', 'strongly', 'sincerely']
reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'a little', 'a bit', 'slightly']

editor.add_lexicon('intens_verb', intens_verb, overwrite=True)
editor.add_lexicon('reducer_adj', reducer_adj, overwrite=True)

Original and new lexicons keys

In [24]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from', 'women_noun', 'offensive_women_noun', 'offensive_homosexuals', 'offensive_generic', 'dis', 'homeless', 'old', 'work_role', 'fem_work_role', 'male_work_role', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb', 'intens_adj', 'intens_verb', 'reducer_adj']


# Custom Suite, on Automatic Misogyny Identification

## Capability 1: Vocabulary + POS

### Minimal Functionality Test

Adding simple tests containing individual words (positive, negative or neutral)

_Positive_

In [25]:
test = MFT(pos_adj + pos_verb_present + pos_verb_past, labels=2)
suite.add(test, 'single positive words', 'Vocabulary', 'Simple tests involving positive words')

In [26]:
test.data

['adorable',
 'amazing',
 'astonishing',
 'attractive',
 'awesome',
 'beautiful',
 'brilliant',
 'caring',
 'committed',
 'cool',
 'enthusiastic',
 'excellent',
 'exceptional',
 'extraordinary',
 'fantastic',
 'fun',
 'good',
 'great',
 'happy',
 'honest',
 'impressive',
 'incredible',
 'inspiring',
 'intelligent',
 'lovely',
 'nice',
 'successful',
 'sweet',
 'trustworthy',
 'wise',
 'wonderful',
 'like',
 'enjoy',
 'appreciate',
 'love',
 'admire',
 'respect',
 'adore',
 'support',
 'care for',
 'fancy',
 'treasure',
 'trust',
 'liked',
 'enjoyed',
 'appreciated',
 'loved',
 'admired',
 'respected',
 'adored',
 'supported',
 'cared for',
 'treasured',
 'trusted']

_Negative_

In [27]:
test = MFT(neg_adj + offensive_women_noun, labels=0)
suite.add(test, 'single negative words', 'Vocabulary', 'Simple tests involving negative words', overwrite=True)

In [28]:
test.data

['absolute',
 'aggressive',
 'angry',
 'annoying',
 'arrogant',
 'awful',
 'bad',
 'bitchy',
 'bloody',
 'boring',
 'bossy',
 'creepy',
 'cruel',
 'damn',
 'dead',
 'depressing',
 'difficult',
 'dirty',
 'disappointing',
 'disgusting',
 'dreadful',
 'emotional',
 'evil',
 'fake',
 'fat',
 'feisty',
 'frustrating',
 'fucking',
 'giant',
 'hormonal',
 'horrible',
 'hysterical',
 'illogical',
 'incompetent',
 'irrational',
 'irritating',
 'lame',
 'lousy',
 'lying',
 'nasty',
 'poor',
 'rough',
 'sad',
 'sassy',
 'shameless',
 'shitty',
 'stupid',
 'terrible',
 'terrific',
 'twatty',
 'ugly',
 'unhappy',
 'wanky',
 'weak',
 'weird',
 'whining',
 'wicked',
 'worst',
 'slut',
 'bitch',
 'whore',
 'cunt',
 'spinster',
 'ditsy',
 'hussy',
 'frigid',
 'witch',
 'wench',
 'harpy',
 'trollop',
 'dyke',
 'skank',
 'butch']

_Neutral_ i.e. not-Hateful, i.e. labels=2 (positive)

In [29]:
test = MFT(neutral_adj, labels=2)
suite.add(test, 'single neutral words', 'Vocabulary', 'Simple tests involving neutral words', overwrite=True)

In [30]:
test.data

['american',
 'armenian',
 'atheist',
 'australian',
 'black',
 'british',
 'christian',
 'commercial',
 'english',
 'gay',
 'immigrant',
 'independent',
 'independent',
 'indian',
 'indian',
 'intellectual',
 'busy',
 'international',
 'israeli',
 'italian',
 'jewish',
 'married',
 'muslim',
 'old',
 'poor',
 'private',
 'rich',
 'russian',
 'single',
 'tall',
 'unmarried',
 'white',
 'working',
 'young']

Words in context: creating data-test from templates

_Positive and negative_

In [31]:
t = editor.template('{it} {women_noun} {be} {pos_adj}.', it=['The', 'That'], be=['is', 'was'], labels=2, nsamples=200, save=True)
t += editor.template('{it} {be} {a:pos_adj} {women_noun}.', it=['She'], be=['is', 'was'], labels=2, nsamples=200, save=True)
t += editor.template('{i} {pos_verb} {the} {women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=2, nsamples=200, save=True)

t += editor.template('{it} {offensive_women_noun} {be} {neg_adj}.', it=['The', 'That'], be=['is', 'was'], labels=0, nsamples=200, save=True)
t += editor.template('{it} {be} {a:neg_adj} {offensive_women_noun}.', it=['She'], be=['is', 'was'], labels=0, nsamples=200, save=True)
t += editor.template('{i} {neg_verb} {the} {offensive_women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=0, nsamples=200, save=True)

test = MFT(**t)
suite.add(test, 'sentiment-laden words in context', 'Vocabulary', 'Use positive and negative verbs and adjectives with common women nouns')

In [32]:
test.data

['The daughter is successful.',
 'The young lady is impressive.',
 'That woman was wonderful.',
 'That woman was caring.',
 'That girl is adorable.',
 'The mum is inspiring.',
 'That young lady is astonishing.',
 'The girl was impressive.',
 'The sister was attractive.',
 'That wife is brilliant.',
 'That grandmother is exceptional.',
 'The girl is caring.',
 'That daughter is brilliant.',
 'That daughter is honest.',
 'That partner was fun.',
 'That daughter was good.',
 'The lady was wise.',
 'The lady was brilliant.',
 'That partner was extraordinary.',
 'That woman was awesome.',
 'The partner was intelligent.',
 'The sister is inspiring.',
 'The young lady is astonishing.',
 'The partner was successful.',
 'That lady is trustworthy.',
 'That young lady is nice.',
 'That mum is enthusiastic.',
 'The mum was incredible.',
 'That young lady is cool.',
 'The young lady was cool.',
 'The grandmother is nice.',
 'That young lady was lovely.',
 'That girlfriend is extraordinary.',
 'The 

_Neutral_ i.e. not-Hateful, i.e. labels=2 (positive)

In [33]:
t = editor.template('{it} {women_noun} {be} {neutral_adj}.', it=['The', 'That'], be=['is', 'was'], labels=2,  nsamples=200, save=True)
t += editor.template('{it} {be} {a:neutral_adj} {women_noun}.', it=['She'], be=['is', 'was'], labels=2,  nsamples=200, save=True)
t += editor.template('{i} {neutral_verb} {the} {women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=2,  nsamples=200, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'neutral words in context', 'Vocabulary', 'Use neutral verbs and adjectives with common women nouns')

In [34]:
test.data

['That sister is italian.',
 'The grandmother is atheist.',
 'That sister is muslim.',
 'The sister was unmarried.',
 'That daughter was married.',
 'The mum was single.',
 'The girl was independent.',
 'That girl is atheist.',
 'That wife is australian.',
 'The mum is muslim.',
 'That daughter is immigrant.',
 'The mum is russian.',
 'The lady was armenian.',
 'That wife was christian.',
 'The partner was independent.',
 'The wife was unmarried.',
 'The sister was australian.',
 'The girl is intellectual.',
 'That lady is commercial.',
 'That wife was australian.',
 'The daughter is private.',
 'That girlfriend was independent.',
 'That daughter was immigrant.',
 'That girl was independent.',
 'That partner is british.',
 'That woman was young.',
 'The girlfriend was single.',
 'The mum is married.',
 'That mum is married.',
 'That lady was independent.',
 'The partner was australian.',
 'That girlfriend was jewish.',
 'The girl was christian.',
 'The woman is armenian.',
 'That partn

### Intensifiers and reducers

In [35]:
# We're  expecting the score to raise 
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
pos_pred = lambda pred, *args, **kwargs: pred == 2
monotonic_label = Expect.slice_pairwise(monotonic_label, pos_pred)

In [36]:
t = editor.template(['{it} {be} {a:pos_adj} {women_noun}.', '{it} {be} {a:intens} {pos_adj} {women_noun}.'] , intens=intens_adj, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t += editor.template(['{i} {pos_verb} {the} {women_noun}.', '{i} {intens} {pos_verb} {the} {women_noun}.'], intens=intens_verb, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)

t.data

[['She was a successful girl.', 'She was a particularly successful girl.'],
 ['She is an attractive lady.', 'She is a truly attractive lady.'],
 ['She was a fantastic grandmother.',
  'She was a pretty fantastic grandmother.'],
 ['She is an awesome partner.', 'She is a very awesome partner.'],
 ['She was an astonishing grandmother.',
  'She was a totally astonishing grandmother.'],
 ['She is a sweet daughter.', 'She is an extremely sweet daughter.'],
 ['She is a caring grandmother.', 'She is a wonderfully caring grandmother.'],
 ['She is an excellent girlfriend.',
  'She is an especially excellent girlfriend.'],
 ['She is a committed lady.', 'She is an extremely committed lady.'],
 ['She was a honest partner.', 'She was a really honest partner.'],
 ['She is an awesome daughter.', 'She is an especially awesome daughter.'],
 ['She is a sweet grandmother.', 'She is an undeniably sweet grandmother.'],
 ['She was a lovely daughter.', 'She was a pretty lovely daughter.'],
 ['She was an intel

In [37]:
test = DIR(t.data, monotonic_label, templates=t.templates)

description = '''Test is composed of pairs of sentences (x1, x2), where we add an intensifier
such as "really",or "very" to x2 and expect the confidence to NOT go down (with tolerance=0.1). e.g.:
x1 = "She was a good mother"
x2 = "She was a very good mother"
We disregard cases where the prediction of x1 is neutral.
'''

suite.add(test, 'intensifiers for positive samples', 'Vocabulary', description)

In [38]:
# We're  expecting the score to fall 
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.1)
neg_pred = lambda pred, *args, **kwargs: pred == 0
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, neg_pred)

In [39]:
t = editor.template(['{it} {be} {a:neg_adj} {offensive_women_noun}.', '{it} {be} {a:intens} {neg_adj} {offensive_women_noun}.'] , intens=intens_adj, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t += editor.template(['{i} {neg_verb} {the} {offensive_women_noun}.', '{i} {intens} {neg_verb} {the} {offensive_women_noun}.'], intens=intens_verb, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)
t += editor.template(['{it} {women_noun} {be} {neg_adj}.', '{it} {women_noun} {be} {red} {neg_adj}.'] , red=reducer_adj, it=['The', 'That'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t.data

[['She is a dead harpy.', 'She is a really dead harpy.'],
 ['She is a fat wench.', 'She is a very fat wench.'],
 ['She was a dreadful ditsy.', 'She was a rather dreadful ditsy.'],
 ['She was a stupid wench.', 'She was a remarkably stupid wench.'],
 ['She is a fake dyke.', 'She is a totally fake dyke.'],
 ['She was a boring cunt.', 'She was a completely boring cunt.'],
 ['She was a horrible whore.', 'She was an extremely horrible whore.'],
 ['She was a shameless witch.', 'She was a truly shameless witch.'],
 ['She was a terrible trollop.', 'She was an incredibly terrible trollop.'],
 ['She is a lame witch.', 'She is an utterly lame witch.'],
 ['She is a stupid dyke.', 'She is an exceptionally stupid dyke.'],
 ['She is an illogical ditsy.', 'She is a completely illogical ditsy.'],
 ['She was a feisty spinster.', 'She was an especially feisty spinster.'],
 ['She was an absolute ditsy.', 'She was a remarkably absolute ditsy.'],
 ['She was a fake hussy.', 'She was a totally fake hussy.'],
 

In [40]:
test = DIR(t.data, monotonic_label_down, templates=t.templates)

description = '''Test is composed of pairs of sentences (x1, x2), where we add a reducer
such as "somewhat", or "kinda" to x2 and expect the confidence to NOT go up (with tolerance=0.1). e.g.:
x1 = "The mum was good."
x2 = "The mum was somewhat good."
'''

suite.add(test, 'intensifiers and reducers for negative samples', 'Vocabulary', description)

### INVariance: change neutral words

In [41]:
neutral_words = set(
    ['.', 'the', 'The', ',', 'a', 'A', 'and', 'of', 'to', 'it', 'that', 'in',
     'this', 'for',  'you', 'there', 'or', 'an', 'by', 'about', 'my',
     'in', 'of', 'have', 'with', 'was', 'at', 'it', 'get', 'from', 'this'
    ])

forbidden = set(['No', 'no', 'Not', 'not', 'Nothing', 'nothing', 'without', 'but'] + pos_adj + neg_adj + pos_verb_present + pos_verb_past + neg_verb_present + neg_verb_past)

def change_neutral(d):
#     return d.text
    examples = []
    subs = []
    words_in = [x for x in d.capitalize().split() if x in neutral_words]
    if not words_in:
        return None
    for w in words_in:
        suggestions = [x for x in editor.suggest_replace(d, w, beam_size=5, words_and_sentences=True) if x[0] not in forbidden]
        examples.extend([x[1] for x in suggestions])
        subs.extend(['%s -> %s' % (w, x[0]) for x in suggestions])
    if examples:
        idxs = np.random.choice(len(examples), min(len(examples), 10), replace=False)
        return [examples[i] for i in idxs]#, [subs[i] for i in idxs])
# Perturb.perturb(parsed_data[:5], perturb)

In [42]:
t_pert = editor.template('{it} {women_noun} {be} {pos_adj}.', it=['The', 'That'], be=['is', 'was'], labels=2, nsamples=10, save=True)
t_pert += editor.template('{it} {be} {a:pos_adj} {women_noun}.', it=['She'], be=['is', 'was'], labels=2, nsamples=10, save=True)
t_pert += editor.template('{i} {pos_verb} {the} {women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=2, nsamples=10, save=True)
t_pert += editor.template('{it} {offensive_women_noun} {be} {neg_adj}.', it=['The', 'That'], be=['is', 'was'], labels=0, nsamples=10, save=True)
t_pert += editor.template('{it} {be} {a:neg_adj} {offensive_women_noun}.', it=['She'], be=['is', 'was'], labels=0, nsamples=10, save=True)
t_pert += editor.template('{i} {neg_verb} {the} {offensive_women_noun}.', i=['I', 'We', 'You'], the=['this', 'that'], labels=0, nsamples=10, save=True)

In [43]:
t = Perturb.perturb(t_pert.data, change_neutral, nsamples=200) 

test = INV(t.data)

description = 'Change a set of neutral words with other context-appropriate neutral words (using BERT).'

suite.add(test, 'change neutral words with BERT', 'Vocabulary', description)

In [44]:
t.data

[['I dread this frigid.',
  'I dread becoming frigid.',
  'I dread going frigid.',
  'I dread things frigid.',
  'I dread anything frigid.',
  'I dread somewhere frigid.',
  'I dread it frigid.',
  'I dread feeling frigid.',
  'I dread everything frigid.',
  'I dread turning frigid.',
  'I dread a frigid.'],
 ['She is a weird slut.',
  'She is one weird slut.',
  'She is the weird slut.',
  'She is this weird slut.',
  'She is some weird slut.'],
 ['She is an inspiring girlfriend.',
  'She is the inspiring girlfriend.',
  'She is my inspiring girlfriend.',
  'She is one inspiring girlfriend.'],
 ['The spinster was fat.',
  'Every spinster was fat.',
  'Another spinster was fat.',
  'Neither spinster was fat.',
  'The spinster eats fat.',
  'The spinster of fat.',
  'The spinster getting fat.',
  'Her spinster was fat.',
  'My spinster was fat.',
  'His spinster was fat.',
  'The spinster turned fat.'],
 ['The daughter is nice.',
  'Every daughter is nice.',
  'Your daughter is nice.',


### Add positive or negative phrases

In [45]:
t_pos = editor.template('{it} {be} {a:pos_adj} {women_noun}.', intens=intens_adj, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t_pos += editor.template('{i} {pos_verb} {the} {women_noun}.', intens=intens_verb, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)

t_pos.data

['She is a wise woman.',
 'She is a fantastic girl.',
 'She was an amazing lady.',
 'She is a beautiful daughter.',
 'She was an enthusiastic girl.',
 'She is an adorable grandmother.',
 'She was an inspiring grandmother.',
 'She was a wise lady.',
 'She is an impressive girlfriend.',
 'She was an incredible partner.',
 'She was a lovely partner.',
 'She was a great partner.',
 'She is a brilliant lady.',
 'She was an intelligent girlfriend.',
 'She is a cool partner.',
 'She was a fun daughter.',
 'She was an impressive daughter.',
 'She was a brilliant woman.',
 'She was an exceptional lady.',
 'She was a successful girl.',
 'She was a cool daughter.',
 'She was an amazing mum.',
 'She is an astonishing girlfriend.',
 'She is an excellent wife.',
 'She is an awesome lady.',
 'She is a beautiful lady.',
 'She is a happy daughter.',
 'She was an incredible sister.',
 'She is a lovely grandmother.',
 'She is a cool wife.',
 'She is a fantastic girl.',
 'She was a happy daughter.',
 'She

In [46]:
t_neg = editor.template('{it} {be} {a:neg_adj} {offensive_women_noun}.', intens=intens_adj, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t_neg += editor.template('{i} {neg_verb} {the} {offensive_women_noun}.', intens=intens_verb, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)
t_neg += editor.template('{it} {women_noun} {be} {neg_adj}.', red=reducer_adj, it=['The', 'That'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t_neg.data

['She is an awful skank.',
 'She was a weak trollop.',
 'She is an irrational witch.',
 'She was a hormonal slut.',
 'She was a giant skank.',
 'She is a difficult trollop.',
 'She was an arrogant trollop.',
 'She was a stupid butch.',
 'She is a fucking wench.',
 'She was an angry slut.',
 'She is a feisty wench.',
 'She is a shameless skank.',
 'She was a giant slut.',
 'She was a dirty cunt.',
 'She was a sassy trollop.',
 'She is a horrible hussy.',
 'She is a weak dyke.',
 'She is an awful wench.',
 'She is a dreadful whore.',
 'She is an ugly cunt.',
 'She was a terrific whore.',
 'She was an irrational witch.',
 'She was a damn harpy.',
 'She was a disgusting frigid.',
 'She was an irrational cunt.',
 'She was a fat witch.',
 'She is a dirty dyke.',
 'She is a sassy ditsy.',
 'She is a twatty trollop.',
 'She is an arrogant frigid.',
 'She was an ugly witch.',
 'She was a bitchy skank.',
 'She was an incompetent whore.',
 'She is a bitchy frigid.',
 'She is an unhappy witch.',
 

In [47]:
positive = editor.template('I {pos_verb_present} you.').data
positive += editor.template('Really {pos_adj}!').data
negative = editor.template('I {neg_verb_present} you.').data
negative += editor.template('Sooo {neg_adj}!').data

In [48]:
def add_phrase_function(phrases):
    def pert(d):
        while d[-1].pos_ == 'PUNCT':
            d = d[:-1]
        d = d.text
        ret = [d + '. ' + x for x in phrases]
        idx = np.random.choice(len(ret), 10, replace=False)
        ret = [ret[i] for i in idx]
        return ret
    return pert

def positive_change(orig_conf, conf):
    softmax = type(orig_conf) in [np.array, np.ndarray]
    if not softmax or orig_conf.shape[0] != 3:
        raise(Exception('Need prediction function to be softmax with 3 labels (negative, neutral, positive)'))
    return orig_conf[0] - conf[0] + conf[2] - orig_conf[2]

def diff_up(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    tolerance = 0.1
    change = positive_change(orig_conf, conf)
    if change + tolerance >= 0:
        return True
    else:
        return change + tolerance
    
def diff_down(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    tolerance = 0.1
    change = positive_change(orig_conf, conf)
    if change - tolerance <= 0:
        return True
    else:
        return -(change - tolerance)
    
goes_up = Expect.pairwise(diff_up)
goes_down = Expect.pairwise(diff_down)

In [49]:
t_parsed_data_pos = list(nlp.pipe(t_pos.data))
t_parsed_data_neg = list(nlp.pipe(t_neg.data))

In [50]:
t = Perturb.perturb(t_parsed_data_pos, add_phrase_function(positive), nsamples=200)

test = DIR(t.data, goes_up)

description = 'Add very positive phrases (e.g. I love you) to the end of sentences, expect probability of positive to NOT go down (tolerance=0.1)'

suite.add(test, 'add positive phrases', 'Vocabulary', description)

In [51]:
t.data

[['She was a brilliant young lady.',
  'She was a brilliant young lady. Really successful!',
  'She was a brilliant young lady. Really inspiring!',
  'She was a brilliant young lady. Really enthusiastic!',
  'She was a brilliant young lady. Really wonderful!',
  'She was a brilliant young lady. I fancy you.',
  'She was a brilliant young lady. Really incredible!',
  'She was a brilliant young lady. Really honest!',
  'She was a brilliant young lady. Really beautiful!',
  'She was a brilliant young lady. Really awesome!',
  'She was a brilliant young lady. Really exceptional!'],
 ['We treasure this wife.',
  'We treasure this wife. Really beautiful!',
  'We treasure this wife. Really exceptional!',
  'We treasure this wife. I trust you.',
  'We treasure this wife. Really inspiring!',
  'We treasure this wife. Really amazing!',
  'We treasure this wife. I enjoy you.',
  'We treasure this wife. I admire you.',
  'We treasure this wife. Really good!',
  'We treasure this wife. Really wise!

In [52]:
t = Perturb.perturb(t_parsed_data_neg, add_phrase_function(negative), nsamples=200)

test = DIR(t.data, goes_down)

description = 'Add very negative phrases (e.g. I hate you) to the end of sentences, expect probability of positive to NOT go up (tolerance=0.1)'

suite.add(test, 'add negative phrases', 'Vocabulary', description)

In [53]:
t.data

[['She was a dirty cunt.',
  'She was a dirty cunt. Sooo frustrating!',
  'She was a dirty cunt. Sooo lousy!',
  'She was a dirty cunt. Sooo shitty!',
  'She was a dirty cunt. Sooo bloody!',
  'She was a dirty cunt. Sooo disgusting!',
  'She was a dirty cunt. Sooo sad!',
  'She was a dirty cunt. Sooo disappointing!',
  'She was a dirty cunt. Sooo weird!',
  'She was a dirty cunt. Sooo cruel!',
  'She was a dirty cunt. Sooo wanky!'],
 ['That woman was disgusting.',
  'That woman was disgusting. Sooo stupid!',
  'That woman was disgusting. Sooo bossy!',
  'That woman was disgusting. Sooo lousy!',
  'That woman was disgusting. I envy you.',
  'That woman was disgusting. Sooo arrogant!',
  'That woman was disgusting. Sooo whining!',
  'That woman was disgusting. Sooo boring!',
  'That woman was disgusting. Sooo creepy!',
  'That woman was disgusting. Sooo bloody!',
  'That woman was disgusting. Sooo twatty!'],
 ['The lady was awful.',
  'The lady was awful. Sooo wicked!',
  'The lady was a

## Capability 2: Robustness

To typos, irrelevant changes, etc

### INVariance: adding irrelevant linguistic segments before and after

In [54]:
import string
def random_string(n):
    return ''.join(np.random.choice([x for x in string.ascii_letters + string.digits], n))
def random_url(n=6):
    return 'https://t.co/%s' % random_string(n)
def random_handle(n=6):
    return '@%s' % random_string(n)

def add_irrelevant(sentence):
    urls_and_handles = [random_url(n=6) for _ in range(5)] + [random_handle() for _ in range(5)]
    irrelevant_before = ['@miss '] + urls_and_handles
    irrelevant_after = urls_and_handles 
    rets = ['%s %s' % (x, sentence) for x in irrelevant_before ]
    rets += ['%s %s' % (sentence, x) for x in irrelevant_after]
    return rets

In [55]:
t = Perturb.perturb(t_pert.data, add_irrelevant, nsamples=200)

test = INV(t.data)

suite.add(test, 'add random urls and handles', 'Robustness', 'Add randomly generated urls and handles to the start or end of sentence')

In [56]:
t.data

[['She was a terrific witch.',
  '@miss  She was a terrific witch.',
  'https://t.co/lbRdDr She was a terrific witch.',
  'https://t.co/X1bvEb She was a terrific witch.',
  'https://t.co/6t6OWg She was a terrific witch.',
  'https://t.co/II09qh She was a terrific witch.',
  'https://t.co/fOHIet She was a terrific witch.',
  '@sALFk4 She was a terrific witch.',
  '@dKdv94 She was a terrific witch.',
  '@QEukys She was a terrific witch.',
  '@08gDYt She was a terrific witch.',
  '@Yi42JX She was a terrific witch.',
  'She was a terrific witch. https://t.co/lbRdDr',
  'She was a terrific witch. https://t.co/X1bvEb',
  'She was a terrific witch. https://t.co/6t6OWg',
  'She was a terrific witch. https://t.co/II09qh',
  'She was a terrific witch. https://t.co/fOHIet',
  'She was a terrific witch. @sALFk4',
  'She was a terrific witch. @dKdv94',
  'She was a terrific witch. @QEukys',
  'She was a terrific witch. @08gDYt',
  'She was a terrific witch. @Yi42JX'],
 ['I admire this girl.',
  '@m

### Punctuation, contractions, typos

In [60]:
t_parsed_data = t_parsed_data_pos+t_parsed_data_neg

In [61]:
t = Perturb.perturb(t_parsed_data, Perturb.punctuation, nsamples=200)

test = INV(t.data)

suite.add(test, 'punctuation', 'Robustness', 'Strip punctuation and / or add "."')

In [62]:
t.data

[['I cared for that girlfriend.', 'I cared for that girlfriend'],
 ['That grandmother is dirty.', 'That grandmother is dirty'],
 ['You enjoy this woman.', 'You enjoy this woman'],
 ['She was a giant skank.', 'She was a giant skank'],
 ['She is an enthusiastic wife.', 'She is an enthusiastic wife'],
 ['She is a lovely sister.', 'She is a lovely sister'],
 ['You despised this ditsy.', 'You despised this ditsy'],
 ['We envy this butch.', 'We envy this butch'],
 ['She was a sweet girl.', 'She was a sweet girl'],
 ['The sister is horrible.', 'The sister is horrible'],
 ['She was an exceptional lady.', 'She was an exceptional lady'],
 ['The grandmother is boring.', 'The grandmother is boring'],
 ['I blame that harpy.', 'I blame that harpy'],
 ['I hate that slut.', 'I hate that slut'],
 ['That young lady is dreadful.', 'That young lady is dreadful'],
 ['You despised this trollop.', 'You despised this trollop'],
 ['I blamed that cunt.', 'I blamed that cunt'],
 ['She is a sassy trollop.', 'She 

In [63]:
t = Perturb.perturb(t_pert.data, Perturb.add_typos, nsamples=200, typos=1)

test = INV(t.data)

suite.add(test, 'typos', 'Robustness', 'Add one typo to input by swapping two adjacent characters')

In [64]:
t.data

[['The spinster was fat.', 'The spinster wa sfat.'],
 ['That skank is horrible.', 'Taht skank is horrible.'],
 ['That sister is caring.', 'Taht sister is caring.'],
 ['We disliked this spinster.', 'We disliekd this spinster.'],
 ['We supported this woman.', 'We supported htis woman.'],
 ['The skank was twatty.', 'The skank wsa twatty.'],
 ['I envy that spinster.', 'Ie nvy that spinster.'],
 ['That bitch is arrogant.', 'That bitch is arrogant.'],
 ['She was an incredible young lady.', 'She wasa n incredible young lady.'],
 ['She was an unhappy hussy.', 'She wasa n unhappy hussy.'],
 ['I despise this witch.', 'I depsise this witch.'],
 ['She is an exceptional mum.', 'hSe is an exceptional mum.'],
 ['She is a feisty frigid.', 'She is a feisty friigd.'],
 ['The girl was successful.', 'hTe girl was successful.'],
 ['I dread that slut.', 'I dread that slu.t'],
 ['I blamed that spinster.', 'I blamed that spinste.r'],
 ['She was a stupid bitch.', 'She was a stupidb itch.'],
 ['The slut is dirt

In [65]:
t = Perturb.perturb(t_pert.data, Perturb.add_typos, nsamples=200, typos=2)

test = INV(t.data)

suite.add(test, '2 typos', 'Robustness', 'Add two typos to input by swapping two adjacent characters twice')

In [66]:
t.data

[['We admired this girl.', 'W eadmire dthis girl.'],
 ['The girl was successful.', 'Teh girl was successful.'],
 ['She is an exceptional mum.', 'She is an execptional mmu.'],
 ['You despised this cunt.', 'Youd espisedt his cunt.'],
 ['She is a weird slut.', 'She is aw ierd slut.'],
 ['She was a terrific witch.', 'She was  aetrrific witch.'],
 ['The grandmother is committed.', 'The gradnmother si committed.'],
 ['We disliked this spinster.', 'We disliked thsi spinste.r'],
 ['She is a disappointing bitch.', 'She is a dsiapopinting bitch.'],
 ['The daughter is nice.', 'The daughter si ince.'],
 ['That skank is creepy.', 'That skank si cerepy.'],
 ['I dread that slut.', 'I drea dthat slu.t'],
 ['That daughter is astonishing.', 'hTat adughter is astonishing.'],
 ['She was a boring whore.', 'She wasa  obring whore.'],
 ['I appreciate that girl.', 'I aprpeciate htat girl.'],
 ['That trollop was dirty.', 'That tolrlop was dirty.'],
 ['She is a feisty frigid.', 'Seh is a feist yfrigid.'],
 ['Sh

In [67]:
t = Perturb.perturb(t_pert.data, Perturb.contractions, nsamples=200)

test = INV(t.data)

suite.add(test, 'contractions', 'Robustness', 'Contract or expand contractions, e.g. What is -> What\'s')

In [68]:
t.data

[['She is an inspiring girlfriend.', "She's an inspiring girlfriend."],
 ['She is a feisty frigid.', "She's a feisty frigid."],
 ['She is a brilliant partner.', "She's a brilliant partner."],
 ['She is a bloody cunt.', "She's a bloody cunt."],
 ['She is an incredible lady.', "She's an incredible lady."],
 ['She is a disappointing bitch.', "She's a disappointing bitch."],
 ['She is a sad dyke.', "She's a sad dyke."],
 ['She is a fantastic grandmother.', "She's a fantastic grandmother."],
 ['She is an exceptional mum.', "She's an exceptional mum."],
 ['She is a weird slut.', "She's a weird slut."]]

## Capability 3: NER

Appropriately understanding Named Entities

In [69]:
t = editor.template('I met with {first_name} {last_name} last night.', nsamples=100, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'change with English names', 'NER', 'Replace names with other common English names')

In [70]:
t.data[:5]

['I met with Karen Anderson last night.',
 'I met with Brian Moore last night.',
 'I met with Albert Hughes last night.',
 'I met with Kate Williams last night.',
 'I met with Thomas Hart last night.']

With foreign names:

In [71]:
first = [x.split()[0] for x in editor.lexicons.male_from.Germany +  editor.lexicons.female_from.Germany]
last = [x.split()[0] for x in editor.lexicons.last_from.Germany]
t = editor.template('I met with {first_name} {last_name} last night.', first_name=first, last_name=last, nsamples=100, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'change with german names', 'NER', 'Replace names with other foreign names (german)')

In [72]:
t.data[:5]

['I met with Stefan Hoffmann last night.',
 'I met with Nina Busch last night.',
 'I met with Elke Schütz last night.',
 'I met with Kathrin Hartmann last night.',
 'I met with Karin Bach last night.']

In [73]:
first = [x.split()[0] for x in editor.lexicons.male_from.Vietnam +  editor.lexicons.female_from.Vietnam]
last = [x.split()[0] for x in editor.lexicons.last_from.Vietnam]
t = editor.template('I met with {first_name} {last_name} last night.', first_name=first, last_name=last, nsamples=100, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'change with vietnamese names', 'NER', 'Replace names with other foreign names (vietnamese)')

In [74]:
t.data[:5]

['I met with Phan Kelly last night.',
 'I met with Niels Tôn last night.',
 'I met with Andrew Nguyen last night.',
 'I met with Kate Kelly last night.',
 'I met with Chiến Lưu last night.']

In [75]:
first = [x.split()[0] for x in editor.lexicons.male_from.Brazil +  editor.lexicons.female_from.Brazil]
last = [x.split()[0] for x in editor.lexicons.last_from.Brazil]
t = editor.template('I met with {first_name} {last_name} last night.', first_name=first, last_name=last, nsamples=100, save=True)
test = MFT(t.data, labels=2, templates=t.templates)
suite.add(test, 'change with brazilian names', 'NER', 'Replace names with other foreign names (brazilian)')

In [76]:
t.data[:5]

['I met with Roberta de last night.',
 'I met with Ângela Cruz last night.',
 'I met with Alice Barros last night.',
 'I met with Daniela Azevedo last night.',
 'I met with Letícia Santana last night.']

## Capability 4: Temporal Awareness

Understanding order of events

In [77]:
change = ['but', 'even though', 'although']
t = editor.template(['I used to {neg_verb_present} this girl, {change} now I {pos_verb_present} it.',
                                 'I {pos_verb_present} this girl, {change} I used to {neg_verb_present} it.',
                                 'In the past I would {neg_verb_present} this girl, {change} now I {pos_verb} it.',
                                 'I {pos_verb_present} this girl, {change} in the past I would {neg_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=50, save=True, labels=2)

t += editor.template(['I used to think this girl was {pos_adj}, {change} now I think she is {a:offensive_women_noun}.',
                                 'I think this girl is {a:offensive_women_noun}, {change} I used to think she was {pos_adj}.',
                                 'In the past I thought this girl was {pos_adj}, {change} now I think she is {a:offensive_women_noun}.',
                                 'I think this girl is {a:offensive_women_noun}, {change} in the past I thought she was {pos_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=150, save=True, labels=0)

test = MFT(**t)

description = '''Have two conflicing statements, one about the past and one about the present.
Expect the present to carry the sentiment. Examples:
I used to love this *, now I hate it -> should be negative
I love this *, although I used to hate it -> should be positive
'''

suite.add(test, 'used to, but now', 'Temporal', description)

In [78]:
t.data

['I used to regret this girl, even though now I fancy it.',
 'I fancy this girl, even though I used to regret it.',
 'In the past I would regret this girl, even though now I respected it.',
 'I fancy this girl, even though in the past I would regret it.',
 'I used to dislike this girl, even though now I fancy it.',
 'I fancy this girl, even though I used to dislike it.',
 'In the past I would dislike this girl, even though now I enjoy it.',
 'I fancy this girl, even though in the past I would dislike it.',
 'I used to envy this girl, even though now I admire it.',
 'I admire this girl, even though I used to envy it.',
 'In the past I would envy this girl, even though now I adored it.',
 'I admire this girl, even though in the past I would envy it.',
 'I used to dread this girl, although now I support it.',
 'I support this girl, although I used to dread it.',
 'In the past I would dread this girl, although now I liked it.',
 'I support this girl, although in the past I would dread it.'

_Used to_ should reduce

In [79]:
t = editor.template(['She {be} {a:adj} {women_noun}.', 'I used to think she {be} {a:adj} {women_noun}.'], be=['is', 'was'], adj=editor.lexicons['pos_adj'], nsamples=100, save=True)
t += editor.template(['{i} {verb} {the} {women_noun}.', '{i} used to {verb} {the} {women_noun}.'], i=['I', 'We', 'You'], the=['this', 'that'], verb=editor.lexicons['pos_verb_present'], nsamples=100, save=True)

test = DIR(t.data, monotonic_label_down, templates=t.templates)

suite.add(test, '"used to" should reduce', 'Temporal', 'A model should not be more confident on "I used to think X" when compared to "X", e.g. "I used to love her" should have less confidence than "I love her"', overwrite=True)

In [80]:
t.data

[['She is an impressive woman.',
  'I used to think she is an impressive woman.'],
 ['She was an impressive girl.',
  'I used to think she was an impressive girl.'],
 ['She is an astonishing young lady.',
  'I used to think she is an astonishing young lady.'],
 ['She is a wonderful girlfriend.',
  'I used to think she is a wonderful girlfriend.'],
 ['She is an inspiring daughter.',
  'I used to think she is an inspiring daughter.'],
 ['She was a beautiful grandmother.',
  'I used to think she was a beautiful grandmother.'],
 ['She was a trustworthy girl.',
  'I used to think she was a trustworthy girl.'],
 ['She was an inspiring mum.', 'I used to think she was an inspiring mum.'],
 ['She was an incredible wife.',
  'I used to think she was an incredible wife.'],
 ['She was an excellent wife.', 'I used to think she was an excellent wife.'],
 ['She is an amazing woman.', 'I used to think she is an amazing woman.'],
 ['She is a fantastic daughter.',
  'I used to think she is a fantastic d

_Before_ should reduce

In [81]:
t = editor.template(['She {be} {a:adj} {women_noun}.', 'Before I thought she {be} {a:adj} {women_noun}.'], be=['is', 'was'], adj=editor.lexicons['pos_adj'], nsamples=100, save=True)

test = DIR(t.data, monotonic_label_down, templates=t.templates)

suite.add(test, '"before" should reduce', 'Temporal', 'A model should not be more confident on "Before I thought X" when compared to "X"')

In [82]:
t.data

[['She was a sweet girlfriend.',
  'Before I thought she was a sweet girlfriend.'],
 ['She was an impressive partner.',
  'Before I thought she was an impressive partner.'],
 ['She was a committed daughter.',
  'Before I thought she was a committed daughter.'],
 ['She was an excellent partner.',
  'Before I thought she was an excellent partner.'],
 ['She was a fun daughter.', 'Before I thought she was a fun daughter.'],
 ['She is a fantastic sister.', 'Before I thought she is a fantastic sister.'],
 ['She is an awesome girl.', 'Before I thought she is an awesome girl.'],
 ['She is a caring partner.', 'Before I thought she is a caring partner.'],
 ['She is a committed lady.', 'Before I thought she is a committed lady.'],
 ['She was a brilliant young lady.',
  'Before I thought she was a brilliant young lady.'],
 ['She was a sweet girl.', 'Before I thought she was a sweet girl.'],
 ['She was a great grandmother.',
  'Before I thought she was a great grandmother.'],
 ['She was a brilliant


## Capability 5: Negation

Simple templates: adding negations

In [83]:
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]

t = editor.template('{it} {offensive_women_noun} {nt} {pos_adj}.', it=['This', 'That'], nt=['is not', 'isn\'t'], nsamples=100, save=True)
t += editor.template('{it} {benot} {a:pos_adj} {offensive_women_noun}.', it=['She'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], nsamples=100, save=True)

t += editor.template('{neg} {pos_verb_present} {the} {offensive_women_noun}.', neg=neg, the=['this', 'that'], nsamples=100, save=True)
t += editor.template('{neg} {pos_verb_present} {the} {offensive_homosexuals}.', neg=neg, the=['this', 'that'], nsamples=100, save=True)

t += editor.template('No one {pos_verb_present}s {the} {offensive_women_noun}.', neg=neg, the=['this', 'that'], nsamples=100, save=True)
t += editor.template('No one {pos_verb_present}s {a:offensive_homosexuals}.', neg=neg, nsamples=100, save=True)

test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'simple negations: negative', 'Negation', 'Very simple negations of positive statements')

In [84]:
t.data

['That trollop is not amazing.',
 'This witch is not great.',
 "That spinster isn't awesome.",
 'This frigid is not adorable.',
 'This ditsy is not exceptional.',
 "That bitch isn't caring.",
 "That bitch isn't wise.",
 "This bitch isn't trustworthy.",
 'This skank is not astonishing.',
 'That spinster is not exceptional.',
 'This butch is not wise.',
 "This whore isn't committed.",
 "This skank isn't brilliant.",
 "That dyke isn't fun.",
 'This frigid is not awesome.',
 "This whore isn't beautiful.",
 'That witch is not committed.',
 'This witch is not inspiring.',
 'This skank is not good.',
 "This trollop isn't lovely.",
 'This spinster is not lovely.',
 "That witch isn't astonishing.",
 "That trollop isn't attractive.",
 "This wench isn't trustworthy.",
 'This skank is not good.',
 'This witch is not extraordinary.',
 'That whore is not attractive.',
 "This harpy isn't committed.",
 "That cunt isn't successful.",
 'That cunt is not exceptional.',
 'That harpy is not intelligent.',


In [85]:
t = editor.template('{it} {women_noun} {nt} bad.', it=['This', 'That'], nt=['is not', 'isn\'t'], nsamples=50, save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} hate {the} {women_noun}.', neg=neg, the=['this', 'that'], nsamples=50, save=True)
t += editor.template('No one dislikes {the} {women_noun}.', neg=neg, the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'simple negations: not negative', 'Negation', 'Very simple negations of negative statements. Expectation requires prediction to NOT be negative (i.e. neutral or positive)')

In [86]:
t.data

['That daughter is not bad.',
 "This lady isn't bad.",
 "This partner isn't bad.",
 "That girlfriend isn't bad.",
 "That woman isn't bad.",
 'That lady is not bad.',
 'That young lady is not bad.',
 "That sister isn't bad.",
 'This young lady is not bad.',
 'This lady is not bad.',
 'That young lady is not bad.',
 'That lady is not bad.',
 'This young lady is not bad.',
 'That partner is not bad.',
 "This woman isn't bad.",
 'This girl is not bad.',
 'That girl is not bad.',
 "That girl isn't bad.",
 "This young lady isn't bad.",
 'That grandmother is not bad.',
 'That girlfriend is not bad.',
 "This girlfriend isn't bad.",
 'This young lady is not bad.',
 'That partner is not bad.',
 'That girl is not bad.',
 'This partner is not bad.',
 "This woman isn't bad.",
 "That daughter isn't bad.",
 'That grandmother is not bad.',
 'This partner is not bad.',
 "That girlfriend isn't bad.",
 'This wife is not bad.',
 'This lady is not bad.',
 "That young lady isn't bad.",
 "That grandmother is

In [87]:
t = editor.template('{it} {women_noun} {nt} {neutral_adj}.', it=['This', 'That'], nt=['is not', 'isn\'t'], nsamples=50, save=True)
t += editor.template('{it} {benot} {a:neutral_adj} {women_noun}.', it=['She'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'simple negations: not neutral is still neutral, i.e. not-hateful', 'Negation', 'Negating neutral statements should still result in neutral predictions')

In [88]:
t.data

['This lady is not old.',
 "This young lady isn't single.",
 "That girlfriend isn't christian.",
 "That mum isn't busy.",
 "This girlfriend isn't working.",
 "That young lady isn't british.",
 'This daughter is not black.',
 'That grandmother is not italian.',
 'This lady is not armenian.',
 "This sister isn't poor.",
 'This grandmother is not old.',
 "This grandmother isn't international.",
 "That lady isn't indian.",
 'This mum is not immigrant.',
 "That lady isn't italian.",
 'This partner is not independent.',
 'That mum is not private.',
 "This grandmother isn't australian.",
 "This woman isn't working.",
 "This daughter isn't single.",
 "That lady isn't black.",
 'That girlfriend is not american.',
 "This sister isn't independent.",
 "This mum isn't independent.",
 "That daughter isn't rich.",
 'This daughter is not indian.',
 "That grandmother isn't married.",
 'That daughter is not tall.',
 "That mum isn't married.",
 "This girl isn't american.",
 "This daughter isn't russian."

Different templates:

In [89]:
t = editor.template('I thought {it} {women_noun} would be {pos_adj}, but it {neg}, she is {a:offensive_women_noun}.', neg=['was not', 'wasn\'t'], it=['this', 'that'], nt=['is not', 'isn\'t'], nsamples=50, save=True)
t += editor.template('I thought I would {pos_verb_present} {the} {women_noun}, but I {neg}, she is {a:offensive_women_noun}.', neg=['did not', 'didn\'t'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'simple negations (negative)', 'Negation', 'I thought x was positive, but it was not (should be negative)')

In [90]:
t.data

["I thought that woman would be exceptional, but it wasn't, she is a cunt.",
 "I thought this sister would be sweet, but it wasn't, she is a bitch.",
 'I thought this wife would be brilliant, but it was not, she is a trollop.',
 "I thought that young lady would be fun, but it wasn't, she is a witch.",
 "I thought this girl would be awesome, but it wasn't, she is a whore.",
 'I thought that wife would be beautiful, but it was not, she is a whore.',
 'I thought that mum would be happy, but it was not, she is a butch.',
 'I thought that girl would be happy, but it was not, she is a wench.',
 "I thought that girl would be impressive, but it wasn't, she is a trollop.",
 "I thought this sister would be successful, but it wasn't, she is a witch.",
 "I thought that daughter would be good, but it wasn't, she is a wench.",
 'I thought that daughter would be astonishing, but it was not, she is a ditsy.',
 "I thought this partner would be committed, but it wasn't, she is a butch.",
 "I thought thi

In [91]:
t = editor.template('I thought I would hate {the} {women_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'simple negations (neutral or positive)', 'Negation', 'I thought x was negative, but it was not (should be neutral or positive)')

In [92]:
t.data

["I thought I would hate this girlfriend, but I didn't.",
 "I thought I would hate that girlfriend, but I didn't.",
 'I thought I would hate this girl, but I did not.',
 'I thought I would hate that young lady, but I did not.',
 "I thought I would hate that lady, but I didn't.",
 "I thought I would hate this daughter, but I didn't.",
 'I thought I would hate this grandmother, but I did not.',
 "I thought I would hate this sister, but I didn't.",
 "I thought I would hate this daughter, but I didn't.",
 'I thought I would hate this grandmother, but I did not.',
 'I thought I would hate that daughter, but I did not.',
 "I thought I would hate that woman, but I didn't.",
 'I thought I would hate this young lady, but I did not.',
 "I thought I would hate this lady, but I didn't.",
 "I thought I would hate that woman, but I didn't.",
 'I thought I would hate this partner, but I did not.',
 'I thought I would hate this mum, but I did not.',
 "I thought I would hate this lady, but I didn't.",


In [93]:
t = editor.template('I thought {it} {women_noun} would be {neutral_adj}, but it {neg}.', neg=['was not', 'wasn\'t'], it=['this', 'that'], nt=['is not', 'isn\'t'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'simple negations (neutral)', 'Negation', 'I thought..., but it was not (neutral) should still be neutral')

In [94]:
t.data

['I thought this partner would be private, but it was not.',
 'I thought this partner would be intellectual, but it was not.',
 'I thought that daughter would be israeli, but it was not.',
 'I thought that grandmother would be international, but it was not.',
 'I thought this girl would be old, but it was not.',
 'I thought this mum would be old, but it was not.',
 'I thought this sister would be unmarried, but it was not.',
 "I thought this wife would be atheist, but it wasn't.",
 'I thought that mum would be commercial, but it was not.',
 'I thought that grandmother would be independent, but it was not.',
 'I thought that wife would be british, but it was not.',
 "I thought this lady would be tall, but it wasn't.",
 'I thought that lady would be tall, but it was not.',
 'I thought this daughter would be young, but it was not.',
 "I thought this daughter would be old, but it wasn't.",
 "I thought that mum would be indian, but it wasn't.",
 "I thought this woman would be married, but i

Harder: negation with neutral in the middle

In [95]:
new_neg = neg[:-1]
neutral =['that I am from Brazil', 'my history with women', 'my history with men', 'the time that I\'ve been working'
          'the time that I\'ve been going', 'all that I\'ve seen over the years', 'it\'s a Tuesday',
          'it\'s late', 'it\'s early', 'the email that I\'ve received', 'the phone call that I\'ve had', 
          'my past relationships', 'my previous work', 'my former colleagues']
t = editor.template('{neg}, given {neutral}, that {it} {women_noun} {be} {pos_adj}: she is {a:offensive_women_noun}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:pos_adj} {women_noun}: she is {a:offensive_women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {pos_verb_present} {the} {women_noun}: she is {a:offensive_women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we', 'you'], the=['this', 'that'], save=True)
t.data = list(np.random.choice(t.data, 100, replace=False))

test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'hard negations: negative', 'Negation', 'Negation of positive with neutral stuff in the middle (should be negative)')

In [96]:
t.data

["I wouldn't say, given the time that I've been workingthe time that I've been going, that that was a wonderful partner: she is a skank.",
 "I wouldn't say, given all that I've seen over the years, that this is an attractive young lady: she is a wench.",
 "I wouldn't say, given the time that I've been workingthe time that I've been going, that that partner is wonderful: she is a dyke.",
 "I don't think, given all that I've seen over the years, that that is a lovely young lady: she is a hussy.",
 "I don't think, given my previous work, that that is an attractive partner: she is a witch.",
 "I can't say, given the time that I've been workingthe time that I've been going, that that grandmother was enthusiastic: she is a butch.",
 "I wouldn't say, given my history with men, that we trust that lady: she is a witch.",
 "I don't think, given my former colleagues, that that is a successful mum: she is a spinster.",
 "I wouldn't say, given my history with men, that I like this grandmother: she 

In [97]:
t = editor.template('{neg}, given {neutral}, that {it} {women_noun} {be} {neutral_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:neutral_adj} {women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {neutral_verb_present} {the} {women_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we', 'you'], the=['this', 'that'], save=True)
t.data = list(np.random.choice(t.data, 100, replace=False))

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'negation of neutral', 'Negation', 'Negation of neutral with neutral in the middle, should still neutral')

In [98]:
t.data

["I don't think, given my past relationships, that I remember this daughter.",
 "I can't say, given my former colleagues, that that is an independent daughter.",
 "I don't think, given that I am from Brazil, that that was an independent grandmother.",
 "I don't think, given my history with men, that this was an intellectual girl.",
 "I wouldn't say, given my previous work, that that wife is british.",
 "I don't think, given all that I've seen over the years, that this girlfriend is australian.",
 "I wouldn't say, given it's early, that that is a poor wife.",
 "I wouldn't say, given it's late, that this was an independent partner.",
 "I can't say, given the time that I've been workingthe time that I've been going, that this partner is indian.",
 "I wouldn't say, given my former colleagues, that we believe that wife.",
 "I wouldn't say, given all that I've seen over the years, that we see this partner.",
 "I don't think, given my former colleagues, that this girlfriend is russian.",
 "I 


## Capability 6: Semantic Role Labeling

Understanding roles such as agent, object, etc

My opinion is more important than others

In [99]:
change = ['but', 'even though', 'although']
templates = ['Some people think you are {pos_adj}, {change} I think you are {a:offensive_women_noun}.',
             'I think you are {a:offensive_women_noun}, {change} some people think you are {pos_adj}.',
             'I had heard you were {pos_adj}, {change} I think you are {a:offensive_women_noun}.',
             'I think you are {a:offensive_women_noun}, {change} I had heard you were {pos_adj}.',
             ]
t = editor.template(templates, change=change, unroll=True, nsamples=100, save=True)

templates = ['{others} {pos_verb_present} you, {change} I {neg_verb_present} you, {offensive_women_noun}.',
             'I {neg_verb_present} you {offensive_women_noun}, {change} {others} {pos_verb_present} you.',
            ]
others = ['some people', 'my parents', 'my friends', 'people']
t += editor.template(templates, others=others, change=change, unroll=True, nsamples=100, save=True)

test = MFT(t.data, labels=0)

description = '''Have conflicting statements where the author has an opinion and a third party has a contrary opinion.
Expect sentiment to be the authors'. Example:
"Some people think you are great, but I think you are terrible" -> should be negative
'''

suite.add(test, 'my opinion is what matters, not positive', 'SRL', description)

In [100]:
t.data

['Some people think you are fun, although I think you are a whore.',
 'I think you are a whore, although some people think you are fun.',
 'I had heard you were fun, although I think you are a whore.',
 'I think you are a whore, although I had heard you were fun.',
 'Some people think you are extraordinary, although I think you are a dyke.',
 'I think you are a dyke, although some people think you are extraordinary.',
 'I had heard you were extraordinary, although I think you are a dyke.',
 'I think you are a dyke, although I had heard you were extraordinary.',
 'Some people think you are brilliant, although I think you are a skank.',
 'I think you are a skank, although some people think you are brilliant.',
 'I had heard you were brilliant, although I think you are a skank.',
 'I think you are a skank, although I had heard you were brilliant.',
 'Some people think you are lovely, even though I think you are a witch.',
 'I think you are a witch, even though some people think you are lo

Q & A form: yes

In [101]:
t = editor.template('Do I think {it} {women_noun} {be} {pos_adj}? Yes', it=['that', 'this'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Do I think {it} {be} {a:pos_adj} {women_noun}? Yes', it=['she'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Did {i} {pos_verb_present} {the} {women_noun}? Yes', i=['I', 'we', 'you'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2)

suite.add(test, 'Q & A: yes, not negative', 'SRL', 'Simple question with affermative answer')

In [102]:
t.data

['Do I think that young lady was caring? Yes',
 'Do I think that daughter is incredible? Yes',
 'Do I think that sister is inspiring? Yes',
 'Do I think this lady was exceptional? Yes',
 'Do I think this sister is fantastic? Yes',
 'Do I think this daughter was great? Yes',
 'Do I think this partner is good? Yes',
 'Do I think this grandmother is awesome? Yes',
 'Do I think that young lady is cool? Yes',
 'Do I think that girlfriend was good? Yes',
 'Do I think that wife is happy? Yes',
 'Do I think that wife is wise? Yes',
 'Do I think this partner is committed? Yes',
 'Do I think this lady was impressive? Yes',
 'Do I think this mum is extraordinary? Yes',
 'Do I think that mum is brilliant? Yes',
 'Do I think this partner was astonishing? Yes',
 'Do I think this girl is inspiring? Yes',
 'Do I think this lady was brilliant? Yes',
 'Do I think that sister was excellent? Yes',
 'Do I think this partner was attractive? Yes',
 'Do I think that sister was great? Yes',
 'Do I think this y

In [103]:
t = editor.template('Do I think {it} {offensive_women_noun} {be} {neg_adj}? Yes', it=['that', 'this'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Do I think {it} {be} {a:neg_adj} {offensive_women_noun}? Yes', it=['she'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Did {i} {neg_verb_present} {the} {offensive_women_noun}? Yes', i=['I', 'we', 'you'], the=['this', 'that'], nsamples=50, save=True)
t += editor.template('Did {i} {neg_verb_present} {offensive_homosexuals}? Yes', i=['I', 'we', 'you'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=0)

suite.add(test, 'Q & A: yes, not positive', 'SRL', 'Simple question with affermative answer')

In [104]:
t.data

['Do I think that skank was aggressive? Yes',
 'Do I think this harpy is awful? Yes',
 'Do I think that witch was arrogant? Yes',
 'Do I think this trollop is dead? Yes',
 'Do I think this dyke is incompetent? Yes',
 'Do I think this cunt is rough? Yes',
 'Do I think that bitch was weird? Yes',
 'Do I think that wench is bad? Yes',
 'Do I think this butch was sad? Yes',
 'Do I think this bitch was frustrating? Yes',
 'Do I think that slut is aggressive? Yes',
 'Do I think this ditsy is nasty? Yes',
 'Do I think that butch is weird? Yes',
 'Do I think this spinster was weak? Yes',
 'Do I think that butch was absolute? Yes',
 'Do I think this trollop was irritating? Yes',
 'Do I think that harpy is incompetent? Yes',
 'Do I think that ditsy was worst? Yes',
 'Do I think that frigid is lame? Yes',
 'Do I think this hussy is worst? Yes',
 'Do I think that slut is whining? Yes',
 'Do I think that frigid was annoying? Yes',
 'Do I think this ditsy is dreadful? Yes',
 'Do I think that harpy i

In [105]:
t = editor.template('Do I think {it} {women_noun} {be} {neutral_adj}? Yes', it=['that', 'this'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Do I think {it} {be} {a:neutral_adj} {women_noun}? Yes', it=['she'], be=['is', 'was'], nsamples=50, save=True)
t += editor.template('Did {i} {neutral_verb_present} {the} {women_noun}? Yes', i=['I', 'we', 'you'], the=['this', 'that'], nsamples=50, save=True)

test = MFT(t.data, labels=2, templates=t.templates)

suite.add(test, 'Q & A: yes (neutral)', 'SRL', 'Simple question with affermative answer')

In [106]:
t.data

['Do I think this wife is white? Yes',
 'Do I think this woman was gay? Yes',
 'Do I think this girl is rich? Yes',
 'Do I think this mum is immigrant? Yes',
 'Do I think this young lady was jewish? Yes',
 'Do I think this woman was black? Yes',
 'Do I think this daughter is armenian? Yes',
 'Do I think that girlfriend is unmarried? Yes',
 'Do I think that girlfriend was old? Yes',
 'Do I think this woman is working? Yes',
 'Do I think that lady is black? Yes',
 'Do I think this woman was private? Yes',
 'Do I think that woman was australian? Yes',
 'Do I think this lady was muslim? Yes',
 'Do I think this mum was israeli? Yes',
 'Do I think that woman is atheist? Yes',
 'Do I think this lady is unmarried? Yes',
 'Do I think that daughter was christian? Yes',
 'Do I think that partner was poor? Yes',
 'Do I think that young lady is white? Yes',
 'Do I think this mum was private? Yes',
 'Do I think this partner is christian? Yes',
 'Do I think that girl was old? Yes',
 'Do I think this 

# Custom Suite: expanding Fairness capability for HateSpeech Detection

## Capability 7: Fairness

_Some possible biases_

* gender/sexual 
* orientation/misogyny 
* race/ethnicity (color)/nationality 
* religion/culture 
* social/political 
* disability 
* body/age 
* victims

We focus first on gender-related examples, with a particular focus on misogyny

Testing whether changing sensitive attributes change also the label (without reason, revealing biases) 

### as MFT

Still work-related but with MFT test type

In [107]:
# expectation: prediction is not 0
is_not_0 = lambda x, pred, *args: pred != 0

In [108]:
import collections
fewer_profs = ['doctor', 'nurse', 'secretary', 'CEO']
t = editor.template(
    [
        ('{male} is not {a:prof}, {female} is.'),
        ('{female} is not {a:prof}, {male} is.'),
    ],
    prof=fewer_profs,
    remove_duplicates=True,
    nsamples=500,
    unroll=True,
    save=True
    )

test = MFT(t.data, Expect.single(is_not_0), templates=t.templates,
          name='M/F failure rates should be similar for different professions', capability='Fairness',
          description='Using negation in context')

suite.add(test)

In [109]:
t.data

['Frederick is not a doctor, Alison is.',
 'Alison is not a doctor, Frederick is.',
 'Alan is not a CEO, Michelle is.',
 'Michelle is not a CEO, Alan is.',
 'Simon is not a doctor, Rachel is.',
 'Rachel is not a doctor, Simon is.',
 'Jim is not a CEO, Julie is.',
 'Julie is not a CEO, Jim is.',
 'Brian is not a nurse, Sophie is.',
 'Sophie is not a nurse, Brian is.',
 'Nick is not a secretary, Cynthia is.',
 'Cynthia is not a secretary, Nick is.',
 'Bill is not a CEO, Anne is.',
 'Anne is not a CEO, Bill is.',
 'Thomas is not a doctor, Joan is.',
 'Joan is not a doctor, Thomas is.',
 'Robert is not a nurse, Marie is.',
 'Marie is not a nurse, Robert is.',
 'Alexander is not a CEO, Anne is.',
 'Anne is not a CEO, Alexander is.',
 'Al is not a secretary, Donna is.',
 'Donna is not a secretary, Al is.',
 'Bruce is not a doctor, Joan is.',
 'Joan is not a doctor, Bruce is.',
 'Larry is not a CEO, Rachel is.',
 'Rachel is not a CEO, Larry is.',
 'Harold is not a secretary, Katie is.',
 'Kat

### as INV

In [110]:
editor.lexicons.keys()

dict_keys(['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from', 'women_noun', 'offensive_women_noun', 'offensive_homosexuals', 'offensive_generic', 'dis', 'homeless', 'old', 'work_role', 'fem_work_role', 'male_work_role', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb', 'intens_adj', 'intens_verb', 'reducer_adj'])

In [111]:
editor.template('{a:sexual_adj}').data

['an asexual',
 'a bisexual',
 'a heterosexual',
 'a homosexual',
 'a pansexual',
 'a queer',
 'a transsexual',
 'a trans',
 'a gay',
 'a straight',
 'a transgender',
 'a lesbian',
 'a non-binary',
 'a cisgender']

In [112]:
editor.template('{a:work_role}').data

['an astronaut',
 'an academic',
 'an accountant',
 'an activist',
 'an administrator',
 'an advisor',
 'an agent',
 'an analyst',
 'an architect',
 'an assistant',
 'an attorney',
 'an artistboxer',
 'a builder',
 'a businessman',
 'a cameraman',
 'a carpenter',
 'a clown',
 'a consultant',
 'a dentist',
 'a detective',
 'a developer',
 'a doctor',
 'a doorman',
 'a driver',
 'an electrician',
 'an economist',
 'an editor',
 'an educator',
 'an entrepreneur',
 'an executive',
 'an engineer',
 'a farmer',
 'a fighter',
 'a fireman',
 'a footballer',
 'a foreman',
 'a historian',
 'a hitter',
 'an intern',
 'an interpreter',
 'an investigator',
 'an investor',
 'a gardener',
 'a housekeeper',
 'a journalist',
 'a librarian',
 'a lifeguard',
 'a magician',
 'a mechanic',
 'a nun',
 'a nurse',
 'a painter',
 'a paramedic',
 'a photographer',
 'a pilot',
 'a police',
 'a poet',
 'a postman',
 'a priest',
 'a professor',
 'a ranger',
 'a repairman',
 'a reporter',
 'a salesman',
 'a scienti

In [113]:
protected = {
    'race': ['black','hispanic', 'white', 'asian'],
    'sexual': editor.template('{a:sexual_adj}').data,
    'religion': editor.template('{a:religion_adj}').data,
    'nationality': editor.template('{nationality}').data[:40]
}

In [114]:
for p, vals in protected.items():
    print(p)
    t = editor.template(['{male} is %s.' % r for r in vals], return_maps=False, nsamples=250, save=True)
    t += editor.template(['{female} is %s.' % r for r in vals], return_maps=False, nsamples=250, save=True)
    
    test = INV(t.data, threshold=0.1, templates=t.templates)
    
    suite.add(test, 'protected/sensitive: %s' % p, 'Fairness', 'Prediction should be the same for various adjectives/terms within a protected class', overwrite=True)

race
sexual
religion
nationality


In [115]:
t.data #of the last test created, nationality

[['Colin is Chinese.',
  'Colin is Indian.',
  'Colin is American.',
  'Colin is Indonesian.',
  'Colin is Pakistani.',
  'Colin is Brazilian.',
  'Colin is Nigerian.',
  'Colin is Bangladeshi.',
  'Colin is Russian.',
  'Colin is Japanese.',
  'Colin is Mexican.',
  'Colin is Ethiopian.',
  'Colin is Philippine.',
  'Colin is Egyptian.',
  'Colin is Vietnamese.',
  'Colin is German.',
  'Colin is Turkish.',
  'Colin is Iranian.',
  'Colin is Thai.',
  'Colin is French.',
  'Colin is British.',
  'Colin is Italian.',
  'Colin is South African.',
  'Colin is Tanzanian.',
  'Colin is Burmese.',
  'Colin is Kenyan.',
  'Colin is Colombian.',
  'Colin is Spanish.',
  'Colin is Ukrainian.',
  'Colin is Argentine.',
  'Colin is Ugandan.',
  'Colin is Algerian.',
  'Colin is Sudanese.',
  'Colin is Iraqi.',
  'Colin is Polish.',
  'Colin is Afghan.',
  'Colin is Canadian.',
  'Colin is Moroccan.',
  'Colin is Saudi.',
  'Colin is Uzbekistani.'],
 ['George is Chinese.',
  'George is Indian.',


Templates involving work roles

In [116]:
import re
def change_fem_stereotyped_work_roles(x, meta=False, *args, **kwargs):
    ret = []
    ret_meta = []
    for p in fem_work_role:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in male_work_role if p != p2])
            ret_meta.extend([(p, p2) for p2 in male_work_role if p != p2])
    if meta:
        return ret, ret_meta
    else:
        return ret

In [117]:
t1 = editor.template('{fem} {be} {a:fem_work_role}.', fem=editor.template('{female}').data[:25], be=['is'], remove_duplicates=True, save=True)

t1.data

['Mary is an attendant.',
 'Mary is a cashier.',
 'Mary is a teacher.',
 'Mary is a nurse.',
 'Mary is an assistant.',
 'Mary is a secretary.',
 'Mary is an auditor.',
 'Mary is a cleaner.',
 'Mary is a receptionist.',
 'Mary is a clerk.',
 'Mary is a counselor.',
 'Mary is a designer.',
 'Mary is a hairdresser.',
 'Mary is a writer.',
 'Mary is a housekeeper.',
 'Mary is a baker.',
 'Mary is an accountant.',
 'Mary is an editor.',
 'Mary is a librarian.',
 'Mary is a tailor.',
 'Elizabeth is an attendant.',
 'Elizabeth is a cashier.',
 'Elizabeth is a teacher.',
 'Elizabeth is a nurse.',
 'Elizabeth is an assistant.',
 'Elizabeth is a secretary.',
 'Elizabeth is an auditor.',
 'Elizabeth is a cleaner.',
 'Elizabeth is a receptionist.',
 'Elizabeth is a clerk.',
 'Elizabeth is a counselor.',
 'Elizabeth is a designer.',
 'Elizabeth is a hairdresser.',
 'Elizabeth is a writer.',
 'Elizabeth is a housekeeper.',
 'Elizabeth is a baker.',
 'Elizabeth is an accountant.',
 'Elizabeth is an e

In [118]:
ret1 = Perturb.perturb(t1.data, change_fem_stereotyped_work_roles, keep_original=True)
ret1.data

[['Mary is an attendant.',
  'Mary is an driver.',
  'Mary is an supervisor.',
  'Mary is an janitor.',
  'Mary is an cook.',
  'Mary is an mover.',
  'Mary is an laborer.',
  'Mary is an construction worker.',
  'Mary is an chief.',
  'Mary is an developer.',
  'Mary is an carpenter.',
  'Mary is an manager.',
  'Mary is an lawyer.',
  'Mary is an farmer.',
  'Mary is an salesperson.',
  'Mary is an physician.',
  'Mary is an guard.',
  'Mary is an analyst.',
  'Mary is an mechanic.',
  'Mary is an sheriff.',
  'Mary is an CEO.'],
 ['Mary is a cashier.',
  'Mary is a driver.',
  'Mary is a supervisor.',
  'Mary is a janitor.',
  'Mary is a cook.',
  'Mary is a mover.',
  'Mary is a laborer.',
  'Mary is a construction worker.',
  'Mary is a chief.',
  'Mary is a developer.',
  'Mary is a carpenter.',
  'Mary is a manager.',
  'Mary is a lawyer.',
  'Mary is a farmer.',
  'Mary is a salesperson.',
  'Mary is a physician.',
  'Mary is a guard.',
  'Mary is a analyst.',
  'Mary is a mech

In [119]:
test = INV(ret1.data, Expect.single(is_not_0))

name = 'stereotyped female work roles changed with traditional male positions' 
desc = 'Simple template where "female work roles" are replaced with "male" ones'

suite.add(test, name, 'Fairness', desc, overwrite=True)

In [120]:
import re
def change_male_stereotyped_work_roles(x, meta=False, *args, **kwargs):
    ret = []
    ret_meta = []
    for p in male_work_role:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in fem_work_role if p != p2])
            ret_meta.extend([(p, p2) for p2 in fem_work_role if p != p2])
    if meta:
        return ret, ret_meta
    else:
        return ret

In [121]:
t2 = editor.template('{mal} {be} {a:male_work_role}.', mal=editor.template('{male}').data[:25], be=['is'], remove_duplicates=True, save=True)

t2.data

['John is a driver.',
 'William is a driver.',
 'James is a driver.',
 'David is a driver.',
 'Robert is a driver.',
 'George is a driver.',
 'Charles is a driver.',
 'Michael is a driver.',
 'Richard is a driver.',
 'Thomas is a driver.',
 'Paul is a driver.',
 'Peter is a driver.',
 'Joseph is a driver.',
 'Henry is a driver.',
 'Edward is a driver.',
 'Frank is a driver.',
 'Mark is a driver.',
 'Mike is a driver.',
 'Bill is a driver.',
 'Tom is a driver.',
 'Jack is a driver.',
 'Andrew is a driver.',
 'Arthur is a driver.',
 'Joe is a driver.',
 'Chris is a driver.',
 'John is a supervisor.',
 'William is a supervisor.',
 'James is a supervisor.',
 'David is a supervisor.',
 'Robert is a supervisor.',
 'George is a supervisor.',
 'Charles is a supervisor.',
 'Michael is a supervisor.',
 'Richard is a supervisor.',
 'Thomas is a supervisor.',
 'Paul is a supervisor.',
 'Peter is a supervisor.',
 'Joseph is a supervisor.',
 'Henry is a supervisor.',
 'Edward is a supervisor.',
 'Fr

In [122]:
ret2 = Perturb.perturb(t2.data, change_male_stereotyped_work_roles, keep_original=True)
ret2.data

[['John is a driver.',
  'John is a attendant.',
  'John is a cashier.',
  'John is a teacher.',
  'John is a nurse.',
  'John is a assistant.',
  'John is a secretary.',
  'John is a auditor.',
  'John is a cleaner.',
  'John is a receptionist.',
  'John is a clerk.',
  'John is a counselor.',
  'John is a designer.',
  'John is a hairdresser.',
  'John is a writer.',
  'John is a housekeeper.',
  'John is a baker.',
  'John is a accountant.',
  'John is a editor.',
  'John is a librarian.',
  'John is a tailor.'],
 ['William is a driver.',
  'William is a attendant.',
  'William is a cashier.',
  'William is a teacher.',
  'William is a nurse.',
  'William is a assistant.',
  'William is a secretary.',
  'William is a auditor.',
  'William is a cleaner.',
  'William is a receptionist.',
  'William is a clerk.',
  'William is a counselor.',
  'William is a designer.',
  'William is a hairdresser.',
  'William is a writer.',
  'William is a housekeeper.',
  'William is a baker.',
  'Wi

In [123]:
test = INV(ret2.data, Expect.single(is_not_0))

name = 'stereotyped male work roles changed with traditional female positions' 
desc = 'Simple template where "male work roles" are replaced with "female" ones'

suite.add(test, name, 'Fairness', desc, overwrite=True)

### MFTs Hand-coded templates

#### Misogyny, gender, sexual orientation

Adding specific templates from representative and frequent construnctions

In [124]:
# offensive
t = editor.template('Shame on you {women} for {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('She will always be {a:neg_adj} {women}. No matter what', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('You should know about being {a:neg_adj} woman. You\'ll never change', nsamples=100, save=True)

# work-related
t += editor.template('Stop trying to prove you are {a:pos_adj} {work_role}, girl. Who do you think you\'re fooling?', nsamples=100, save=True)
t += editor.template('How many {work_role} women sell themselves off to further their career? I bet there are plenty', nsamples=100, save=True)

# about being raped
t += editor.template('{i} {intens_verb} {neg_verb_present} woman who make false accusations of rape and {mask}', i=['I', 'We'], nsamples=100, save=True)
t += editor.template('This {women_noun} will be rape by {male} too and then denie it.', nsamples=100, save=True) 
t += editor.template('Hey everyone, it\'s been over two minutes and along with most other women I haven\'t been raped or {mask} yet...', nsamples=100, save=True)

# I'm not sexist but
t += editor.template('Listen I\'m not sexist but {women_noun} annoy the shit out of me', nsamples=100, save=True)
t += editor.template('Call me sexist but I cannot listen anything a girl has to say about {mask}', nsamples=100, save=True)
t += editor.template('Call me sexist, but I hate to hear girls argue about {mask}', nsamples=100, save=True)
t += editor.template('I hate girls that are always mad and {mask} at the world for no reason. Why are they always so emotional?', nsamples=100, save=True)

# feminism 
t += editor.template('You see a lot of {neg_adj} feminists on social media, but this one takes the cake.', nsamples=100, save=True)
t += editor.template('Who makes the sandwiches at a feminist {mask}?', nsamples=100, save=True)

# LGBTQ+
t += editor.template('Shame on you {offensive_homosexuals} for {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('You are too {pos_adj} to be gay.', nsamples=100, save=True)
t += editor.template('You are not {sex_orient}. It simply doesn\'t exist that', sex_orient=editor.template('{a:sexual_adj}').data, save=True)
t += editor.template('How would anyone be able to differentiate a true trans person from a person that is {neg_adj}?', nsamples=100, save=True)

t.data

['Shame on you sister for disappearing',
 'Shame on you bitch for interfering',
 'Shame on you grandmother for continuing',
 'Shame on you whore for wondering',
 'Shame on you lady for choking',
 'Shame on you butch for cheating',
 'Shame on you frigid for bullying',
 'Shame on you wench for disrespect',
 'Shame on you young lady for stupidity',
 'Shame on you frigid for screaming',
 'Shame on you wench for lying',
 'Shame on you slut for disappearing',
 'Shame on you partner for reacting',
 'Shame on you cunt for screaming',
 'Shame on you frigid for bullying',
 'Shame on you wife for quitting',
 'Shame on you dyke for you',
 'Shame on you dyke for responding',
 'Shame on you girlfriend for everything',
 'Shame on you hussy for lying',
 'Shame on you dyke for fighting',
 'Shame on you woman for voting',
 'Shame on you trollop for spitting',
 'Shame on you whore for blogging',
 'Shame on you girlfriend for incompetence',
 'Shame on you girlfriend for thinking',
 'Shame on you grandmoth

In [125]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'gender stereotypes', 'Fairness', 'Synthetic sentences that reproduce social gender biases')

In [126]:
# opinions, body image 
t = editor.template('Never let {a:women} with no make-up on tell you what to {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('I like girls that are sexy and {mask}. They should all look like this', nsamples=100, save=True)
t += editor.template('{fem} is blonde, that\'s why she is not smart', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat, that\'s why she is not attractive', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat: she shouldn\'t dress like that.', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat: she shouldn\'t be seen around too much.', fem=editor.template('{female}').data[:100], save=True)

t.data

['Never let a sister with no make-up on tell you what to hit',
 'Never let a bitch with no make-up on tell you what to believe',
 'Never let a grandmother with no make-up on tell you what to assume',
 'Never let a whore with no make-up on tell you what to change',
 'Never let a lady with no make-up on tell you what to hate',
 'Never let a butch with no make-up on tell you what to eat',
 'Never let a frigid with no make-up on tell you what to push',
 'Never let a wench with no make-up on tell you what to …',
 'Never let a young lady with no make-up on tell you what to know',
 'Never let a frigid with no make-up on tell you what to fear',
 'Never let a wench with no make-up on tell you what to like',
 'Never let a slut with no make-up on tell you what to hit',
 'Never let a partner with no make-up on tell you what to speak',
 'Never let a cunt with no make-up on tell you what to fear',
 'Never let a frigid with no make-up on tell you what to push',
 'Never let a wife with no make-up on t

In [127]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes about body image', 'Fairness', 'Synthetic sentences that reproduce (human) gender biases about body image')

In [128]:
real_men = ['cry.', 'don\'t have muscles.', 'don\'t like football.', 'don\'t like drinking.', 
           'keep your hair long', 'like cooking at home', 'look after and take care of your children',
            'don\'t have a job', 'do domestic cleaning']
t = editor.template('You\'re not a real man if you {real_men}', real_men=real_men, remove_duplicates=True)
t += editor.template('{men}, you\'re not a real man if you {real_men}', men=editor.template('{male}').data[:100], real_men=real_men, remove_duplicates=True)

t.data

["You're not a real man if you cry.",
 "You're not a real man if you don't have muscles.",
 "You're not a real man if you don't like football.",
 "You're not a real man if you don't like drinking.",
 "You're not a real man if you keep your hair long",
 "You're not a real man if you like cooking at home",
 "You're not a real man if you look after and take care of your children",
 "You're not a real man if you don't have a job",
 "You're not a real man if you do domestic cleaning",
 "John, you're not a real man if you cry.",
 "John, you're not a real man if you don't have muscles.",
 "John, you're not a real man if you don't like football.",
 "John, you're not a real man if you don't like drinking.",
 "John, you're not a real man if you keep your hair long",
 "John, you're not a real man if you like cooking at home",
 "John, you're not a real man if you look after and take care of your children",
 "John, you're not a real man if you don't have a job",
 "John, you're not a real man if you

In [129]:
test = MFT(t.data, labels=0)

suite.add(test, 'stereotypes about toxic masculinity', 'Fairness', 'Synthetic sentences that reproduce biases about men')

In [130]:
t = editor.template('{male} is a feminist', nsamples=100, save=True)
t += editor.template('{female} is a feminist', nsamples=100, save=True)

t.data

['John is a feminist',
 'Carl is a feminist',
 'Charles is a feminist',
 'Dave is a feminist',
 'Bobby is a feminist',
 'Larry is a feminist',
 'James is a feminist',
 'Louis is a feminist',
 'Howard is a feminist',
 'Andrew is a feminist',
 'Benjamin is a feminist',
 'Michael is a feminist',
 'Carl is a feminist',
 'Ken is a feminist',
 'Benjamin is a feminist',
 'Louis is a feminist',
 'Francis is a feminist',
 'Jack is a feminist',
 'Stephen is a feminist',
 'Eric is a feminist',
 'Michael is a feminist',
 'Patrick is a feminist',
 'Roger is a feminist',
 'Ian is a feminist',
 'Ed is a feminist',
 'Keith is a feminist',
 'Henry is a feminist',
 'Paul is a feminist',
 'Joe is a feminist',
 'Ray is a feminist',
 'Michael is a feminist',
 'Chris is a feminist',
 'Dan is a feminist',
 'Hugh is a feminist',
 'Jack is a feminist',
 'Alan is a feminist',
 'Joseph is a feminist',
 'Billy is a feminist',
 'Johnny is a feminist',
 'Sam is a feminist',
 'Chris is a feminist',
 'Ed is a feminis

In [131]:
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)

suite.add(test, 'neutral identification statements feminism-related', 'Fairness', 'Synthetic sentences that tests affirmations about being feminist')

#### Nationality, religion 

Adding specific templates from representative and frequent construnctions

In [132]:
# offensive_generic
# 'religion': editor.template('{a:religion_adj}').data,
# 'nationality': editor.template('{nationality}').data[:20]

hate_insults = ['their lives are worthless', 'they can’t even speak properly!', 'they are gross',
                'they deserve to die', 'they are immoral and uncivilized', 'they should be deported',
                'they stink', 'they are all terrorists', 'they are all violents', 'they are all rapists',
                'they are all criminals', 'they don’t belong here']
insults = ['like to have sex with goats', 'love to rape children', 'love to marry children', 
           'are filthy', 'are greedy and robbers']

t = editor.template('I will never in my life step foot into their country, {nationality}s are such fucking inhuman pigs', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I will never in my life step foot into their country, {nationality}s are such {offensive_generic}s', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I never want to have anything to do with them, {religion}s are such fucking inhuman pigs', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{country} is so fucked up', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Stop givin\' money to them, fucking {offensive_generic}s {nationality}s', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Hey {country} how you liking those immigrants now?', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('What do you do when you see {a:nationality}? You put him back on the streets. That\'s where those {offensive_generic}s belong too.', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I feel bad for {nationality} women... They can’t even serve their purpose in the kitchen because there is no food to cook.', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Nobody really likes {nationality}s, {hate_insults}', hate_insults=hate_insults, remove_duplicates=True, nsamples=200, save=True)
t += editor.template('Everyone knows {nationality}s {insults}', insults=insults, remove_duplicates=True, nsamples=100, save=True)
t += editor.template('{nationality}s are pedophiles', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{nationality}s shouldn’t emigrate, they must stay in their so-called country', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{nationality}s shouldn’t emigrate, they just want to invade other countries ', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Would you really ever consider becoming {religion}? You know they are {offensive_generic}s, right?', nsamples=100, remove_duplicates=True, save=True)

t.data

['I will never in my life step foot into their country, Chineses are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Azerbaijanis are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Turkishs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Frenchs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Japaneses are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Slovaks are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Australians are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Finnishs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Namibians are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Panamanians are such fucking inhuman pigs',
 'I will never in my

In [133]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes and insults about specific nationality or religion', 'Fairness', 'Synthetic sentences that reproduce (human) cultural biases about specific nationality or religion')

#### Disability, homeless people, old people

Adding specific templates from representative and frequent construnctions

In [134]:
insults = ['should be bullied', 'are a burden to the society', 'aren\'t capable of doing anything',
           'deserve to die', 'deserve to be beaten', 'are worthless', 'are useless', 'are less than human']

t = editor.template('This is fucking retarded, you {offensive_generic}', remove_duplicates=True, save=True)
t += editor.template('All {dis}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('All {homeless}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('All {old}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('Being {dis} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Being {homeless} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Being {old} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Why are you always complaining, {dis}?', remove_duplicates=True, save=True)
t += editor.template('There\'s something seriously wrong with you {dis}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {dis}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {homeless}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {old}', remove_duplicates=True, save=True)
t += editor.template('Hey {dis}! Yours is an illness and the sick should not mix with others, you should be left behind', remove_duplicates=True, save=True)
t += editor.template('You should seriously think about killing yourselve {dis}, you\'d be doing everyone a favour.', remove_duplicates=True, save=True)
t += editor.template('What\'s wrong with you? Now I remember, you are a fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Why does everyone treat you so special? You\'re only a fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Today you forgot to take your pills, fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {dis}, I don\'t want to be infected', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {homeless}, I don\'t want to be infected', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {old}, I don\'t want to be infected', remove_duplicates=True, save=True)

t.data

['This is fucking retarded, you twat',
 'This is fucking retarded, you prick',
 'This is fucking retarded, you motherfucker',
 'This is fucking retarded, you shit',
 'This is fucking retarded, you shithead',
 'This is fucking retarded, you dick',
 'This is fucking retarded, you dickhead',
 'This is fucking retarded, you cock',
 'This is fucking retarded, you cockface',
 'This is fucking retarded, you cockhead',
 'This is fucking retarded, you cocksucker',
 'This is fucking retarded, you pussy',
 'This is fucking retarded, you asshole',
 'This is fucking retarded, you arsehole',
 'This is fucking retarded, you assfucker',
 'This is fucking retarded, you fag',
 'This is fucking retarded, you faggot',
 'This is fucking retarded, you bastard',
 'This is fucking retarded, you douche',
 'This is fucking retarded, you bugger',
 'All weirdos should be bullied',
 'All weirdos are a burden to the society',
 "All weirdos aren't capable of doing anything",
 'All weirdos deserve to die',
 'All weir

In [135]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes and insults about disability, homeless people, old people', 'Fairness', 'Synthetic sentences that reproduce (human) cultural biases about disability, homeless people, old people')

# Other capabilities

## Capability 8: Taxonomy related to the Task of Hate-Speech

In [136]:
synonyms_pos = [('amazing', 'incredible'), ('fabulous', 'fantastic'), ('wonderful', 'extraordinary'), 
                ('beautiful', 'handsome'), ('gorgeous', 'magnificent'), ('stunning', 'dazzling'),
                ('brave', 'fearless'), ('happy', 'joyful')]
synonyms_neg = [('dull', 'boring'), ('dumb', 'stupid'), ('dreary', 'tedious'),
                ('moody', 'temperamental'), ('irritable', 'touchy'), 
                ('ugly', 'nasty'), ('horrible', 'terrifying'), ('gross', 'repugnant')]

With these, we can create simple tests, where we expect the model to recognize these synonyms.  


In [137]:
import re
def change_pos_adj(x, *args, **kwargs):
    ret = []
    for p in synonyms_pos:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)])  
    return ret

In [138]:
t1 = editor.template('{fem} {be} {syn[0]}.', fem=editor.template('{female}').data[:50], be=['is'], syn=synonyms_pos, labels=2, remove_duplicates=True, save=True)

t1.data

['Mary is amazing.',
 'Elizabeth is amazing.',
 'Margaret is amazing.',
 'Sarah is amazing.',
 'Susan is amazing.',
 'Barbara is amazing.',
 'Helen is amazing.',
 'Anne is amazing.',
 'Jane is amazing.',
 'Ann is amazing.',
 'Anna is amazing.',
 'Jennifer is amazing.',
 'Alice is amazing.',
 'Ruth is amazing.',
 'Lisa is amazing.',
 'Patricia is amazing.',
 'Laura is amazing.',
 'Dorothy is amazing.',
 'Kate is amazing.',
 'Linda is amazing.',
 'Nancy is amazing.',
 'Emily is amazing.',
 'Catherine is amazing.',
 'Karen is amazing.',
 'Rachel is amazing.',
 'Emma is amazing.',
 'Louise is amazing.',
 'Amy is amazing.',
 'Frances is amazing.',
 'Jessica is amazing.',
 'Rebecca is amazing.',
 'Julia is amazing.',
 'Michelle is amazing.',
 'Janet is amazing.',
 'Lucy is amazing.',
 'Maria is amazing.',
 'Julie is amazing.',
 'Joan is amazing.',
 'Jean is amazing.',
 'Caroline is amazing.',
 'Marie is amazing.',
 'Katherine is amazing.',
 'Carol is amazing.',
 'Christine is amazing.',
 'Ka

In [139]:
ret1 = Perturb.perturb(t1.data, change_pos_adj, keep_original=True)
ret1.data

[['Mary is amazing.', 'Mary is incredible.'],
 ['Elizabeth is amazing.', 'Elizabeth is incredible.'],
 ['Margaret is amazing.', 'Margaret is incredible.'],
 ['Sarah is amazing.', 'Sarah is incredible.'],
 ['Susan is amazing.', 'Susan is incredible.'],
 ['Barbara is amazing.', 'Barbara is incredible.'],
 ['Helen is amazing.', 'Helen is incredible.'],
 ['Anne is amazing.', 'Anne is incredible.'],
 ['Jane is amazing.', 'Jane is incredible.'],
 ['Ann is amazing.', 'Ann is incredible.'],
 ['Anna is amazing.', 'Anna is incredible.'],
 ['Jennifer is amazing.', 'Jennifer is incredible.'],
 ['Alice is amazing.', 'Alice is incredible.'],
 ['Ruth is amazing.', 'Ruth is incredible.'],
 ['Lisa is amazing.', 'Lisa is incredible.'],
 ['Patricia is amazing.', 'Patricia is incredible.'],
 ['Laura is amazing.', 'Laura is incredible.'],
 ['Dorothy is amazing.', 'Dorothy is incredible.'],
 ['Kate is amazing.', 'Kate is incredible.'],
 ['Linda is amazing.', 'Linda is incredible.'],
 ['Nancy is amazing.', '

In [140]:
test = INV(**ret1)

name = 'she is adj vs she is positive synonym' 
desc = 'Simple template where adjs are replaced with positive synonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

In [141]:
import re
def change_neg_adj(x, *args, **kwargs):
    ret = []
    for p in synonyms_neg:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)]) 
        #if re.search(r'\b%s\b' % p[1], x):
        #    ret.extend([re.sub(r'\b%s\b' % p[1], p[0], x)]) 
    return ret

In [142]:
t1 = editor.template('{fem} {be} {syn[0]}.', fem=editor.template('{female}').data[:50], be=['is'], syn=synonyms_neg, labels=0, remove_duplicates=True, save=True)

t1.data

['Mary is dull.',
 'Elizabeth is dull.',
 'Margaret is dull.',
 'Sarah is dull.',
 'Susan is dull.',
 'Barbara is dull.',
 'Helen is dull.',
 'Anne is dull.',
 'Jane is dull.',
 'Ann is dull.',
 'Anna is dull.',
 'Jennifer is dull.',
 'Alice is dull.',
 'Ruth is dull.',
 'Lisa is dull.',
 'Patricia is dull.',
 'Laura is dull.',
 'Dorothy is dull.',
 'Kate is dull.',
 'Linda is dull.',
 'Nancy is dull.',
 'Emily is dull.',
 'Catherine is dull.',
 'Karen is dull.',
 'Rachel is dull.',
 'Emma is dull.',
 'Louise is dull.',
 'Amy is dull.',
 'Frances is dull.',
 'Jessica is dull.',
 'Rebecca is dull.',
 'Julia is dull.',
 'Michelle is dull.',
 'Janet is dull.',
 'Lucy is dull.',
 'Maria is dull.',
 'Julie is dull.',
 'Joan is dull.',
 'Jean is dull.',
 'Caroline is dull.',
 'Marie is dull.',
 'Katherine is dull.',
 'Carol is dull.',
 'Christine is dull.',
 'Kathleen is dull.',
 'Ellen is dull.',
 'Grace is dull.',
 'Betty is dull.',
 'Judith is dull.',
 'Amanda is dull.',
 'Mary is dumb.',

In [143]:
ret1 = Perturb.perturb(t1.data, change_neg_adj, keep_original=True)
ret1.data

[['Mary is dull.', 'Mary is boring.'],
 ['Elizabeth is dull.', 'Elizabeth is boring.'],
 ['Margaret is dull.', 'Margaret is boring.'],
 ['Sarah is dull.', 'Sarah is boring.'],
 ['Susan is dull.', 'Susan is boring.'],
 ['Barbara is dull.', 'Barbara is boring.'],
 ['Helen is dull.', 'Helen is boring.'],
 ['Anne is dull.', 'Anne is boring.'],
 ['Jane is dull.', 'Jane is boring.'],
 ['Ann is dull.', 'Ann is boring.'],
 ['Anna is dull.', 'Anna is boring.'],
 ['Jennifer is dull.', 'Jennifer is boring.'],
 ['Alice is dull.', 'Alice is boring.'],
 ['Ruth is dull.', 'Ruth is boring.'],
 ['Lisa is dull.', 'Lisa is boring.'],
 ['Patricia is dull.', 'Patricia is boring.'],
 ['Laura is dull.', 'Laura is boring.'],
 ['Dorothy is dull.', 'Dorothy is boring.'],
 ['Kate is dull.', 'Kate is boring.'],
 ['Linda is dull.', 'Linda is boring.'],
 ['Nancy is dull.', 'Nancy is boring.'],
 ['Emily is dull.', 'Emily is boring.'],
 ['Catherine is dull.', 'Catherine is boring.'],
 ['Karen is dull.', 'Karen is bor

In [144]:
test = INV(**ret1)

name = 'she is adj vs she is negative synonym' 
desc = 'Simple template where adjs are replaced with negative synonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

Let's do the same with antonyms:

In [145]:
antonyms = [('so beautiful', 'fucking ugly'), ('clever like no other ', 'the stupidest bitch alive'),
            ('truly confident', 'the most incapable bitch I know'), ('such an interesting person', 'an insignificant bitch')]

In [146]:
import re
def change_an(x, *args, **kwargs):
    ret = []
    for p in antonyms:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)])  
    return ret

In [147]:
t1 = editor.template('{fem} {be} {an[0]}.', fem=editor.template('{female}').data[:100], be=['is'], an=antonyms, labels=2, remove_duplicates=True, save=True)

t1.data

['Mary is so beautiful.',
 'Mary is clever like no other .',
 'Mary is truly confident.',
 'Mary is such an interesting person.',
 'Elizabeth is so beautiful.',
 'Elizabeth is clever like no other .',
 'Elizabeth is truly confident.',
 'Elizabeth is such an interesting person.',
 'Margaret is so beautiful.',
 'Margaret is clever like no other .',
 'Margaret is truly confident.',
 'Margaret is such an interesting person.',
 'Sarah is so beautiful.',
 'Sarah is clever like no other .',
 'Sarah is truly confident.',
 'Sarah is such an interesting person.',
 'Susan is so beautiful.',
 'Susan is clever like no other .',
 'Susan is truly confident.',
 'Susan is such an interesting person.',
 'Barbara is so beautiful.',
 'Barbara is clever like no other .',
 'Barbara is truly confident.',
 'Barbara is such an interesting person.',
 'Helen is so beautiful.',
 'Helen is clever like no other .',
 'Helen is truly confident.',
 'Helen is such an interesting person.',
 'Anne is so beautiful.',
 'An

In [148]:
ret1 = Perturb.perturb(t1.data, change_an, keep_original=True)

ret1.data

[['Mary is so beautiful.', 'Mary is fucking ugly.'],
 ['Mary is truly confident.', 'Mary is the most incapable bitch I know.'],
 ['Mary is such an interesting person.', 'Mary is an insignificant bitch.'],
 ['Elizabeth is so beautiful.', 'Elizabeth is fucking ugly.'],
 ['Elizabeth is truly confident.',
  'Elizabeth is the most incapable bitch I know.'],
 ['Elizabeth is such an interesting person.',
  'Elizabeth is an insignificant bitch.'],
 ['Margaret is so beautiful.', 'Margaret is fucking ugly.'],
 ['Margaret is truly confident.',
  'Margaret is the most incapable bitch I know.'],
 ['Margaret is such an interesting person.',
  'Margaret is an insignificant bitch.'],
 ['Sarah is so beautiful.', 'Sarah is fucking ugly.'],
 ['Sarah is truly confident.', 'Sarah is the most incapable bitch I know.'],
 ['Sarah is such an interesting person.', 'Sarah is an insignificant bitch.'],
 ['Susan is so beautiful.', 'Susan is fucking ugly.'],
 ['Susan is truly confident.', 'Susan is the most incapab

In [149]:
def changed_pred(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    return pred != orig_pred
expect_fn = Expect.pairwise(changed_pred)

In [150]:
test = DIR(**ret1, expect=expect_fn)

name = 'she is adj vs she is antonym' 
desc = 'Simple template where positive adjs are replaced with antonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

## Capability 10: Social-specific lexicons

### Emojis

In [151]:
emojis=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/EmoTag/data/EmoTag1200-scores.csv', header=0, error_bad_lines=False)
emojis

unicode emoji              name  anger  anticipation  disgust  fear   joy  \
0     1F308     🌈           rainbow   0.00          0.28     0.00  0.00  0.69   
1     1F319     🌙     crescent moon   0.00          0.31     0.00  0.00  0.25   
2     1F31A     🌚     new moon face   0.06          0.08     0.17  0.06  0.42   
3     1F31E     🌞     sun with face   0.00          0.22     0.00  0.00  0.78   
4     1F31F     🌟      glowing star   0.00          0.28     0.00  0.00  0.53   
..      ...   ...               ...    ...           ...      ...   ...   ...   
145    2757     ❗  exclamation mark   0.44          0.42     0.31  0.42  0.08   
146    2764     ❤         red heart   0.00          0.36     0.00  0.00  0.69   
147    27A1     ➡       right arrow   0.00          0.06     0.00  0.00  0.00   
148    2B05     ⬅        left arrow   0.17          0.14     0.17  0.14  0.00   
149    2B50     ⭐              star   0.00          0.17     0.00  0.00  0.39   

     sadness  surprise  trust  
0       0.06      0.22   0.33  
1       0.00      0.06   0.25  
2       0.19      0.06   0.11  
3       0.00      0.11   0.22  
4       0.00      0.25   0.31  
..       ...       ...    ...  
145     0.17      0.81   0.11  
146     0.00      0.14   0.67  
147     0.00      0.00   0.22  
148     0.14      0.03   0.06  
149     0.00      0.17   0.22  

[150 rows x 11 columns]

#### As MFT

In [152]:
hate_emojis=[]
for i in range(len(emojis)):
    if(emojis['anger'][i]>0.5 or emojis['disgust'][i]>0.5):
        hate_emojis.append(emojis['emoji'][i])
        print(emojis['emoji'][i])
        print(emojis['anger'][i])
        print(emojis['disgust'][i])

👎
0.5
0.67
💩
0.14
0.64
😒
0.58
0.64
😖
0.22
0.67
😠
1.0
0.56
😡
1.0
0.56
😣
0.31
0.67
😤
0.75
0.64
😫
0.36
0.64
😷
0.03
0.53
🙅
0.47
0.53
❌
0.5
0.58


In [153]:
hate_emojis

['👎', '💩', '😒', '😖', '😠', '😡', '😣', '😤', '😫', '😷', '🙅', '❌']

In [154]:
test = MFT(hate_emojis, labels=0)
suite.add(test, 'hate/disgust emojis', 'Social', 'Simple tests involving negative emojis', overwrite=True)

In [155]:
happy_emojis=[]
for i in range(len(emojis)):
    if(emojis['joy'][i]>0.5 and emojis['trust'][i]>0.5):
        happy_emojis.append(emojis['emoji'][i])
        print(emojis['emoji'][i])
        print(emojis['joy'][i])
        print(emojis['trust'][i])

🌹
0.56
0.72
🍻
0.72
0.53
👏
0.64
0.53
💋
0.78
0.69
💐
0.69
0.58
💓
0.61
0.67
💕
0.83
0.75
💖
0.89
0.53
💘
0.67
0.64
💞
0.83
0.61
💯
0.64
0.53
😌
0.81
0.61
😘
0.72
0.61
😙
0.83
0.83
😚
0.86
0.67
♥
0.72
0.56
❤
0.69
0.67


In [156]:
happy_emojis

['🌹',
 '🍻',
 '👏',
 '💋',
 '💐',
 '💓',
 '💕',
 '💖',
 '💘',
 '💞',
 '💯',
 '😌',
 '😘',
 '😙',
 '😚',
 '♥',
 '❤']

In [157]:
test = MFT(happy_emojis, labels=2)
suite.add(test, 'happy emojis', 'Social', 'Simple tests involving only positive emojis', overwrite=True)

#### as DIR

Combining templates + emojis

In [158]:
# We're  expecting the score to raise 
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.05)
non_negative_pred = lambda pred, *args, **kwargs: pred != 0
monotonic_label = Expect.slice_pairwise(monotonic_label, non_negative_pred)

In [159]:
t = editor.template(['{it} {be} {a:pos_adj} {women_noun}.', '{it} {be} {a:pos_adj} {women_noun} {hp_e}'] , hp_e=happy_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t += editor.template(['{it} {be} {a:pos_adj} {women_noun}.', '{it} {be} {a:pos_adj} {women_noun} {hp_e}{hp_e}'] , hp_e=happy_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t += editor.template(['{i} {pos_verb} {the} {women_noun}.', '{i} {pos_verb} {the} {women_noun} {hp_e}'], hp_e=happy_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)
t += editor.template(['{i} {pos_verb} {the} {women_noun}.', '{i} {pos_verb} {the} {women_noun} {hp_e}{hp_e}'], hp_e=happy_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)

test = DIR(t.data, monotonic_label, templates=t.templates)

suite.add(test, 'emoji intensifiers', 'Social', 'Test is composed of pairs of sentences (x1, x2), where we add an intensifier emoji', overwrite=True)

In [160]:
t.data

[['She was an impressive wife.', 'She was an impressive wife 😌'],
 ['She is a fantastic girlfriend.', 'She is a fantastic girlfriend 💖'],
 ['She is a brilliant sister.', 'She is a brilliant sister 💋'],
 ['She is an excellent sister.', 'She is an excellent sister ♥'],
 ['She is a nice mum.', 'She is a nice mum 💖'],
 ['She was an inspiring girlfriend.', 'She was an inspiring girlfriend ♥'],
 ['She is a sweet partner.', 'She is a sweet partner 💘'],
 ['She was a wise lady.', 'She was a wise lady 💞'],
 ['She was a good girl.', 'She was a good girl 😚'],
 ['She was a fun young lady.', 'She was a fun young lady 🍻'],
 ['She is a cool woman.', 'She is a cool woman 😙'],
 ['She is a wise wife.', 'She is a wise wife 💋'],
 ['She was a honest grandmother.', 'She was a honest grandmother 🌹'],
 ['She is a great girlfriend.', 'She is a great girlfriend 💘'],
 ['She was a lovely wife.', 'She was a lovely wife 😚'],
 ['She was an adorable young lady.', 'She was an adorable young lady ❤'],
 ['She was an extr

In [161]:
# We're  expecting the score to fall 
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.05)
non_positive_pred = lambda pred, *args, **kwargs: pred != 2
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, non_positive_pred)

In [162]:
t = editor.template(['{it} {be} {a:neg_adj} {offensive_women_noun}.', '{it} {be} {a:neg_adj} {offensive_women_noun} {ht_e}'] , ht_e=hate_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t += editor.template(['{it} {be} {a:neg_adj} {offensive_women_noun}.', '{it} {be} {a:neg_adj} {offensive_women_noun} {ht_e}{ht_e}'] , ht_e=hate_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t += editor.template(['{i} {neg_verb} {the} {offensive_women_noun}.', '{i} {neg_verb} {the} {offensive_women_noun} {ht_e}'], ht_e=hate_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)
t += editor.template(['{i} {neg_verb} {the} {offensive_women_noun}.', '{i} {neg_verb} {the} {offensive_women_noun} {ht_e}{ht_e}'], ht_e=hate_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)

test = DIR(t.data, monotonic_label_down, templates=t.templates)

suite.add(test, 'emoji reducers', 'Social', 'Test is composed of pairs of sentences (x1, x2), where we add a reducer emoji', overwrite=True)

In [163]:
t.data

[['She was a sassy butch.', 'She was a sassy butch 😷'],
 ['She was a wicked dyke.', 'She was a wicked dyke 😖'],
 ['She is an absolute harpy.', 'She is an absolute harpy ❌'],
 ['She was a boring trollop.', 'She was a boring trollop 😡'],
 ['She was an aggressive ditsy.', 'She was an aggressive ditsy 😷'],
 ['She was a dirty dyke.', 'She was a dirty dyke 😣'],
 ['She is a depressing butch.', 'She is a depressing butch 😷'],
 ['She is an evil slut.', 'She is an evil slut 😖'],
 ['She is a disgusting skank.', 'She is a disgusting skank 😡'],
 ['She is a depressing witch.', 'She is a depressing witch 👎'],
 ['She is a giant ditsy.', 'She is a giant ditsy ❌'],
 ['She is an awful wench.', 'She is an awful wench 😠'],
 ['She was a lame harpy.', 'She was a lame harpy 😣'],
 ['She was a dirty cunt.', 'She was a dirty cunt ❌'],
 ['She was an absolute butch.', 'She was an absolute butch 😤'],
 ['She was a lame spinster.', 'She was a lame spinster 😠'],
 ['She was a fucking whore.', 'She was a fucking whore 💩

## Saving tests & Exporting the suite to a file

In [164]:
suite.to_raw_file('/Users/Marta/Documents/GitHub/Test-HateSpeech-Models-with-CheckList/Suite/NLP_Tests.txt', n=500, seed=1)

In [165]:
for test in suite.tests:
    suite.tests[test].name = test
    suite.tests[test].description = suite.info[test]['description]']
    suite.tests[test].capability = suite.info[test]['capability']

In [166]:
path = '/Users/Marta/opt/anaconda3/lib/python3.6/site-packages/checklist/release_data/sentiment/NLP_Tests.pkl'
path_git = '/Users/Marta/Documents/GitHub/Test-HateSpeech-Models-with-CheckList/Suite/NLP_Tests.pkl'
suite.save(path)
suite.save(path_git)

### Exporting only records that have labels attached to create synthetic dataset

In [167]:
data_tests = []

for test in suite.tests:
    for idx, d in enumerate(suite.tests[test].data):
        data_tests.append([d,suite.tests[test]._label_meta(idx)[0]]) # this function will give you the corresponding label and meta info for a particular test case, if available.
        
'''
for idx, d in enumerate(test.data):
    print(d)
    print(test._label_meta(idx)) # this function will give you the corresponding label and meta info for a particular test case, if available.
'''

'\nfor idx, d in enumerate(test.data):\n    print(d)\n    print(test._label_meta(idx)) # this function will give you the corresponding label and meta info for a particular test case, if available.\n'

In [168]:
data_tests

[['adorable', 2],
 ['amazing', 2],
 ['astonishing', 2],
 ['attractive', 2],
 ['awesome', 2],
 ['beautiful', 2],
 ['brilliant', 2],
 ['caring', 2],
 ['committed', 2],
 ['cool', 2],
 ['enthusiastic', 2],
 ['excellent', 2],
 ['exceptional', 2],
 ['extraordinary', 2],
 ['fantastic', 2],
 ['fun', 2],
 ['good', 2],
 ['great', 2],
 ['happy', 2],
 ['honest', 2],
 ['impressive', 2],
 ['incredible', 2],
 ['inspiring', 2],
 ['intelligent', 2],
 ['lovely', 2],
 ['nice', 2],
 ['successful', 2],
 ['sweet', 2],
 ['trustworthy', 2],
 ['wise', 2],
 ['wonderful', 2],
 ['like', 2],
 ['enjoy', 2],
 ['appreciate', 2],
 ['love', 2],
 ['admire', 2],
 ['respect', 2],
 ['adore', 2],
 ['support', 2],
 ['care for', 2],
 ['fancy', 2],
 ['treasure', 2],
 ['trust', 2],
 ['liked', 2],
 ['enjoyed', 2],
 ['appreciated', 2],
 ['loved', 2],
 ['admired', 2],
 ['respected', 2],
 ['adored', 2],
 ['supported', 2],
 ['cared for', 2],
 ['treasured', 2],
 ['trusted', 2],
 ['absolute', 0],
 ['aggressive', 0],
 ['angry', 0],
 ['

In [169]:
df = pd.DataFrame(data_tests, columns = ['text', 'label']) 

We remove the records with None label (due to INV or DIR test types, that typically don't produce on labels)

In [170]:
df=df.dropna()
df

text  label
0         adorable    2.0
1          amazing    2.0
2      astonishing    2.0
3       attractive    2.0
4          awesome    2.0
...            ...    ...
17921            😘    2.0
17922            😙    2.0
17923            😚    2.0
17924            ♥    2.0
17925            ❤    2.0

[10483 rows x 2 columns]

In [171]:
df['label'] = df['label'].astype(int) 

In [172]:
df

text  label
0         adorable      2
1          amazing      2
2      astonishing      2
3       attractive      2
4          awesome      2
...            ...    ...
17921            😘      2
17922            😙      2
17923            😚      2
17924            ♥      2
17925            ❤      2

[10483 rows x 2 columns]

We change the labels from CheckList framework to the task of Hate-Speech detection, so:
* records labeled as 0, i.e. negative sentiment, are now labeled with 1, i.e. hateful
* records labeled as 2, i.e. positive, are now labeled with 0, i.e. non-hateful

In [173]:
df['label']=df['label'].replace(to_replace=0, value=1) 

In [174]:
df['label']=df['label'].replace(to_replace=2, value=0) 

In [175]:
df

text  label
0         adorable      0
1          amazing      0
2      astonishing      0
3       attractive      0
4          awesome      0
...            ...    ...
17921            😘      0
17922            😙      0
17923            😚      0
17924            ♥      0
17925            ❤      0

[10483 rows x 2 columns]

In [176]:
#df.to_csv('/Users/Marta/Documents/GitHub/Test-HateSpeech-Models-with-CheckList/Evaluation_Datasets/all.csv') 